In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.chrome.options import Options
import os
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException
from IPython.display import display, clear_output
import ipywidgets as widgets
import tkinter as tk
from tkinter import filedialog
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton
import sys
import re
import zipfile #extract zip file
import requests
from bs4 import BeautifulSoup
PATH = "/Users/frrookie521/Desktop/civil/cee495/cee495/Google\ Chrome\ for\ Testing.app"
download_path = "/Users/frrookie521/Desktop/civil/cee495/"
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
prefs = {
        'download.default_directory': download_path,
        'download.prompt_for_download': False,
        'download.directory_upgrade': True,
        'safebrowsing.enabled': True
         }
chrome_options.add_experimental_option('prefs', prefs)
excel_file = pd.read_excel('VariableSources_2.xlsx', header=None)
driver = webdriver.Chrome(options=chrome_options)
driver.set_window_size(1396, 1000)
urls = excel_file.iloc[1:, 2].tolist() #store urls from third column and second row 
unique = set(url for url in urls if isinstance(url, str)) #extact the unique urls from excel
zipfiles = ['file1.zip', 'file2.zip'] 
def text_to_csv(input_filename, output_filename): #convert txt file to csv 
    with open(input_filename, 'r') as txt: #open the file
        lines = txt.readlines() #read all the lines 
    with open(output_filename, 'w') as csv_file: #set up the csv file 
        for line in lines:
            fields = line.strip().split()  # split the line
            csv_line = ','.join(fields)    # Join fields with commas for CSV
            csv_file.write(csv_line + '\n')
def census_geographic(driver, unique):
    driver.get(url)
    time.sleep(5)
    select = driver.find_element(By.ID, 'toolbar-button-datasets')
    select.click()
    time.sleep(3)
    year_5 = driver.find_element(By.XPATH, '//*[@id="table-header"]/div[2]/div[2]/div/div/div/div[2]/ul/div/li[2]/div[1]/div/div')
    year_5.click()
    wait = WebDriverWait(driver, 10)
    geo = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="toolbar-button-geos"]')))
    geo.click()
    wait = WebDriverWait(driver,10)
    county = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[2]/div/div/div[1]/div/div[2]/div/div/div/section/div[2]/div[2]/div/div[3]')))
    county.click()
    time.sleep(5)
    Clicklist = driver.find_element(By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[2]/div/div/div[1]/div/div[2]/div/div/div/section/ul/li[1]/div/div[1]/div[2]')
    Clicklist.click()
    cancel = driver.find_element(By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[2]/div/div/div[1]/div/div[1]/div/div[1]/div[1]/div/div[4]')
    cancel.click()
    wait = WebDriverWait(driver,15)
    download = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[1]/div[2]/div[2]/div/div/div[4]/div/button')))
    download.click()
    time.sleep(3)
    year_2020= driver.find_element(By.ID, 'Table: S0701 Vintage: 2020')
    year_2020.click()
    time.sleep(1)
    year_2019= driver.find_element(By.XPATH, '/html/body/div/div/div[3]/div/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div[5]/div/div')
    year_2019.click()
    time.sleep(1)
    elements_2018= driver.find_elements(By.ID, 'Table: S0701 Vintage: 2018') #there are two same ID 
    if len(elements_2018) >= 2:
        for element in elements_2018:
            if element == elements_2018[1]:
                element.click()
                break
    time.sleep(1)
    elements_2017 = driver.find_elements(By.ID, 'Table: S0701 Vintage: 2017') #there are two same IDs
    if len(elements_2017) >= 2:
        for Element in elements_2017:
            if Element == elements_2017[1]:
                Element.click()
                break
    time.sleep(1)            
    download_csv = driver.find_element(By.XPATH, '//*[@id="app"]/div[3]/div/div/div[2]/div[2]/div[4]/div[2]/button')
    download_csv.click()
    time.sleep(10)
    attempts = 0
    max_attempts = 5  # Maximum number of attempts 
    time_break = 10
    while attempts < max_attempts: #trying to do the mechanism 
        files = os.listdir(download_path)
        zipfiles = [file for file in files if file.endswith('.zip')]
        if zipfiles:
            for zip_file in zipfiles:
                zip_file_path = os.path.join(download_path, zip_file)
                with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                    zip_ref.extractall(download_path)
                os.remove(zip_file_path)
            break  # Break out of the retry loop if zip files are found and extracted successfully
        else:
            attempts += 1
            if attempts < max_attempts:
                print(f"No ZIP files found in the download_path. wait {time_break}second for retrying...")
                time.sleep(time_break)
            else:
                print("No ZIP files found after maximum attempts.")
    time.sleep(10)
    years = range(2017, 2021)
    for year in years:
        csv_file = f'ACSST5Y{year}.S0701-Data.csv'
        csv_path = os.path.join(download_path, csv_file)
        table = pd.read_csv(csv_path, header=None, dtype=str)
        table.drop(0, axis=0, inplace=True)
        table.columns = table.iloc[0]
        table = table[1:]
        columns_keyword = []
        for column in table.columns:
            if isinstance(column, str) and ("!!Moved" in column or "Geo" in column) and ("Annotation" not in column and "Margin" not in column):
                columns_keyword.append(column)  
        selected_columns = table[columns_keyword]
        selected_columns.to_csv(f'{year}.csv', index=False)
def census_demographics(driver,unique):
    driver.get(unique)
    time.sleep(5)
    select = driver.find_element(By.ID, 'toolbar-button-datasets')
    select.click()
    time.sleep(3)
    year_5 = driver.find_element(By.XPATH, '//*[@id="app"]/div[3]/div/div/div[2]/div[2]/div[4]/div[2]/button')
    year_5.click()
    wait = WebDriverWait(driver, 10)
    geo = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="toolbar-button-geos"]')))
    geo.click()
    wait = WebDriverWait(driver,20)
    county = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[2]/div/div/div[1]/div/div[2]/div/div/div/section/div[2]/div[2]/div/div[3]')))
    county.click()
    time.sleep(5)
    Clicklist = driver.find_element(By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[2]/div/div/div[1]/div/div[2]/div/div/div/section/ul/li[1]/div/div[1]/div[2]')
    Clicklist.click()
    cancel = driver.find_element(By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[2]/div/div/div[1]/div/div[1]/div/div[1]/div[1]/div/div[4]')
    cancel.click()
    wait = WebDriverWait(driver,15)
    download = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[1]/div[2]/div[2]/div/div/div[4]/div/button')))
    download.click()
    time.sleep(3)
    year_2020= driver.find_element(By.ID, 'Table: DP05 Vintage: 2020')
    year_2020.click()
    time.sleep(1)
    year_2019= driver.find_elements(By.ID, 'Table: DP05 Vintage: 2019')
    if len(year_2019) >= 2:
        for element in year_2019:
            if element == year_2019[1]:
                element.click()
                break
    time.sleep(1)
    elements_2018= driver.find_elements(By.ID, 'Table: DP05 Vintage: 2018') #there are two same ID 
    if len(elements_2018) >= 2:
        for element in elements_2018:
            if element == elements_2018[1]:
                element.click()
                break
    time.sleep(1)
    elements_2017 = driver.find_elements(By.ID, 'Table: DP05 Vintage: 2017') #there are two same IDs
    if len(elements_2017) >= 2:
        for Element in elements_2017:
            if Element == elements_2017[1]:
                Element.click()
                break
    time.sleep(1)            
    download_csv = driver.find_element(By.XPATH, '//*[@id="app"]/div[3]/div/div/div[2]/div[2]/div[4]/div[2]/button')
    download_csv.click()
    time.sleep(20)
    attempts = 0
    max_attempts = 5  # Maximum number of attempts 
    time_break = 10
    while attempts < max_attempts: #trying to do the mechanism 
        files = os.listdir(download_path)
        zipfiles = [file for file in files if file.endswith('.zip')]
        if zipfiles:
            for zip_file in zipfiles:
                zip_file_path = os.path.join(download_path, zip_file)
                with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                    zip_ref.extractall(download_path)
                os.remove(zip_file_path)
            break  # Break out of the retry loop if zip files are found and extracted successfully
        else:
            attempts += 1
            if attempts < max_attempts:
                print(f"No ZIP files found in the download_path. wait {time_break}second for retrying...")
                time.sleep(time_break)
            else:
                print("No ZIP files found after maximum attempts.")
def tamu_umr_report(driver,unique):
    driver.get(unique) 
    download_csv = driver.find_element(By.XPATH, '//*[@id="genesis-content"]/article/div/p[2]/a[1]')
    download_csv.click()
    time.sleep(10)
    excelfile = 'complete-data-2021-umr-by-tti.xlsx'
    excel_extracted = pd.read_excel(excelfile)
    excel_extracted
    combined_labels = excel_extracted.iloc[:4].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=0) #combine first 4 row as one label 
    excel_extracted = excel_extracted.iloc[4:] #replace first 4 row name
    excel_extracted.loc[-1] = combined_labels
    excel_extracted.index = excel_extracted.index + 1
    excel_extracted.sort_index(inplace=True)
    excel_extracted.rename(columns={'Unnamed: 23': 'Annual Hours of Delay'}, inplace=True) #renamed column name 
    columns = ['Area Group', 'Urban Area', 'Primary','Population','Year','Arterial Street','Annual Hours of Delay']
    selected = excel_extracted[columns]
    csv_file_path = 'umr.csv'
    selected.to_csv(csv_file_path, index=False)
    os.remove('complete-data-2021-umr-by-tti.xlsx')
def census_data(driver,unique):
    driver.get(unique) 
    download_csv=driver.find_element(By.ID,'anch_33')
    download_csv.click()
    time.sleep(10)
    table = pd.read_excel('bfs_county_apps_annual.xlsx', header= None)
    table = table.iloc[2:]
    selected = table
    csv_file_path = 'Business Applications by County.csv'
    selected.to_csv(csv_file_path, index=False)
    os.remove('bfs_county_apps_annual.xlsx')
def census_survey(driver,unique):
    driver.get(unique)
    time.sleep(5)
    year_2021 = driver.find_element(By.XPATH, '//*[@id="listArticlesContainer_List_1222676053"]/div[2]/a[1]')
    year_2021.click()
    time.sleep(5)
    county_2021 = driver.find_element(By.XPATH, '//*[@id="listArticlesContainer_List_309820028"]/div[2]/div[2]/div/span/a')
    county_2021.click()
    attempts = 0
    max_attempts = 5  # Maximum number of attempts 
    time_break = 10
    while attempts < max_attempts: #trying to do the mechanism 
        files = os.listdir(download_path)
        zipfiles = [file for file in files if file.endswith('.zip')]
        if zipfiles:
            for zip_file in zipfiles:
                zip_file_path = os.path.join(download_path, zip_file)
                with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                    zip_ref.extractall(download_path)
                os.remove(zip_file_path)
            break  # Break out of the retry loop if zip files are found and extracted successfully
        else:
            attempts += 1
            if attempts < max_attempts:
                print(f"No ZIP files found in the download_path. wait {time_break}second for retrying...")
                time.sleep(time_break)
            else:
                print("No ZIP files found after maximum attempts.")
    time.sleep(10)
    input_text_file = 'cbp21co.txt'
    output_csv_file = 'cbp21co.csv'
    text_to_csv(input_text_file, output_csv_file)
def cnbc_business(driver,unique):
    current_year = datetime.datetime.now().year  #get the current year
    for year in range(2007, current_year + 1): #we make for loop for each year 
        filtered_columns = [] #we define the empty filtered column 
        if year in [2009, 2013]: #because 2009 and 2013 overall ranking have unique html comparing to other 
            if year == 2009:
                url = "https://www.cnbc.com/id/100000992"
            elif year == 2013: #create additional condition
                url = "https://www.cnbc.com/id/100824779"
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')
            table = soup.find('table')
        else: #other years 
            if year >= 2014:
                for month in range(6,8):
                    for day in range(10,24):
                        url = f"https://www.cnbc.com/{year}/{month}/{day}/americas-top-states-for-business.html"
                        response = requests.get(url)
                        if response.status_code == 200: #if that url is exisiting, code is 200
                            soup = BeautifulSoup(response.content, 'html.parser')
                            table = soup.find('table')
                            break
                    else:
                        continue #creating iteration for checking the html 
                    break 
            else:
                url = f"https://www.cnbc.com/top-states-{year}-overall-rankings/" #the url linkfor overall rankings
                response = requests.get(url)
                soup = BeautifulSoup(response.content, 'html.parser')
                table = soup.find('table')
        if table:
            column_names = []
            data_rows = []
            for i, tr in enumerate(table.find_all('tr')):
                if i == 0:  # Header row
                    th_tags = tr.find_all(['th', 'td'])
                    column_names = [th.text.strip() for th in th_tags if th.text.strip()]
                else:
                    row = [td.text.strip() for td in tr.find_all('td')]
                    if row:
                        data_rows.append(row)
       
            overall_labels = ["Overall Rank", "Overall","Rank","Overall Ranking"] #multiple labels
            costofbusiness_labels = ["Cost of Business", "Cost of Doing Business"]
            business_columns = ["Business Friendliness","Business Friendly"]
            Infra = ["Infrastructure","Transportation","Infrastructure & Transp.","Infrastructure &amp; Transp."] #does include transportation??
            State = ["State"]
            table_extract = pd.DataFrame(data_rows, columns=column_names)
            table_extract.rename(columns=dict(zip(overall_labels, ['Overall Ranking']*len(overall_labels))), inplace=True)  # Rename into standard form
            table_extract.rename(columns=dict(zip(costofbusiness_labels, ['Cost of Doing Business']*len(costofbusiness_labels))), inplace=True)  # Rename into standard form
            table_extract.rename(columns=dict(zip(Infra, ['Infrastructure']*len(Infra))), inplace=True)  # Rename into standard form
            table_extract.rename(columns=dict(zip(business_columns, ['Business Friendliness']*len(business_columns))), inplace=True)  # Rename into standard form
            for label in overall_labels:
                if label in table_extract.columns:
                    filtered_columns.insert(0, label)
                    #rename into standard form 
                    break
            for label in costofbusiness_labels:
                if label in table_extract.columns:
                    filtered_columns.insert(0, label)
                    break
            for label in business_columns:
                if label in table_extract.columns:
                    filtered_columns.insert(0, label)
                    break
            for label in Infra:
                if label in table_extract.columns:
                    filtered_columns.insert(0, label)
                    break
            for label in State:
                if label in table_extract.columns:
                    filtered_columns.insert(0, label)
                    break
            filtered_data = table_extract[filtered_columns]
            filtered_data.to_csv(f'extracted{year}.csv', index=False, header=True, mode='w')
            print(f" {year} downloaded.")
def census_time(driver,unique):
    driver.get(unique)
    time.sleep(10)
    html = "https://www2.census.gov/programs-surveys/bds/tables/time-series/bds2020_st_cty.csv"
    driver.get(html)
def realtor_research(driver,unique):
    url_historical = 'https://econdata.s3-us-west-2.amazonaws.com/Reports/Core/RDC_Inventory_Core_Metrics_County_History.csv'
    driver.get(url_historical)
    time.sleep(70)
    table = pd.read_csv('RDC_Inventory_Core_Metrics_County_History.csv')
    table.rename(columns={'month_date_yyyymm': 'year&month'}, inplace=True)
    table.rename(columns={'county_name': 'County'}, inplace=True)
    table.rename(columns={'county_fips': 'County Federal Information Processing Standard'}, inplace=True)
    table.rename(columns={'median_square_feet': 'median listing square feet'}, inplace=True)
    columns = ['year&month', 'County', 'County Federal Information Processing Standard','median listing square feet']
    selected = table[columns]
    csv_file_path = 'realtor.csv'
    selected.to_csv(csv_file_path, index=False)
    os.remove('RDC_Inventory_Core_Metrics_County_History.csv')
def taxfoundation(driver,unique):
    url = 'https://taxfoundation.org/2023-state-business-tax-climate-index/' #the url for 2007 overall raking
    response = requests.get(url)  #get the webpage
#require beautifulsoup to extract html documents
    Object = BeautifulSoup(response.content, 'html.parser')
    table = Object.find('table') #locate the table in the html
    column_names = []
    first_row = table.find('tr')
    for th in first_row.find_all(['th', 'td']):
        column_name = th.text.strip()
        if column_name:
            column_names.append(column_name)
    data_rows = []  #we initalize the data row number 
    for tr in table.find_all('tr'): #tr represent each number 
        row = [td.text.strip() for td in tr.find_all('td')] #we finds all the row elements within the current row element and create a new list containing the text content of each row element in the row
        if row: #use to check whether there is empty row 
            data_rows.append(row) #if the row is not empty, then we show it in the data_row
    table_tax = pd.DataFrame(data_rows, columns=column_names)
    table_tax.to_csv('table_tax.csv', index=False)
def census_construction(driver,unique):
    base = 'https://www2.census.gov/econ/bps/County/'
    driver.get(base)
    time.sleep(5)
    def process_txt_content(txt_content, output_csv_file):
        lines = txt_content.split('\n')
        with open(output_csv_file, 'w') as csv_file:
            for line in lines:
                fields = line.strip().split()  # Adjust if needed
                csv_line = ','.join(fields)
                csv_file.write(csv_line + '\n')
    fourth_digit = 1
    third_digit = 0   #from year 2018 
    stop_loop = False #we define the stop loop for the entire iteration 
    generated_filenames = set()
    while fourth_digit < 10 and not stop_loop:
        for last_two_digits in range(1, 14):  # The last two digits are changing from 1 to 13
            if fourth_digit == 2 and third_digit == 3 and last_two_digits == 7:
                if file_type == 'c':
                    stop_loop = True
                    driver.quit()
                    break 
            if last_two_digits > 12:
                third_digit += 1
                if third_digit == 10:
                    fourth_digit += 1
                    third_digit = 0
                adjusted_last_digits = last_two_digits - 12 
                adjusted = True
            else:
                adjusted_last_digits = last_two_digits
                adjusted = False  
            for file_type in ['c', 'y']:
                if fourth_digit == 2 and third_digit == 3 and last_two_digits == 7 and file_type == 'c':
                    stop_loop = True
                    driver.quit()
                    break
                else:
                    file_name = f"co{str(fourth_digit)}{str(third_digit)}{str(adjusted_last_digits).zfill(2)}{file_type}.txt"
                    if file_name in generated_filenames:
                        continue
                    generated_filenames.add(file_name) 
                    full_url = base + file_name
                    driver.get(full_url)
                    driver.refresh()
                    time.sleep(2)
                    response = requests.get(full_url)
                    if response.status_code == 200:
                        content = response.text
                        local_file_path = f"co{str(fourth_digit)}{str(third_digit)}{str(adjusted_last_digits).zfill(2)}{file_type}.txt"
                        with open(local_file_path, "w") as file:
                            file.write(content)
                        table = pd.read_csv(local_file_path, sep=',')
                        table.reset_index(inplace=True)
                        table.drop(columns=['FIPS.1', 'Region', 'Unnamed: 6', 'Unnamed: 9', 'Unnamed: 12', 'Unnamed: 15', 'Unnamed: 17', 'Unnamed: 18', '1-unit rep', 'Unnamed: 20', 'Unnamed: 21', '2-units rep', 'Unnamed: 23', 'Unnamed: 24', '3-4 units rep', 'Unnamed: 26', 'Unnamed: 27', ' 5+units rep'], inplace=True)
                        table.rename(columns={'Survey': 'State', 'FIPS': 'County', 'Division': 'County Name', 'County': 'total1Unit', '1-unit': 'value1Unit', 'Unnamed: 8': 'total2Unit', '2-units': 'value2Unit', 'Unnamed: 11': 'total34Unit', '3-4 units': 'value34Unit', 'Unnamed: 14': 'total5Unit', '5+ units': 'value5Unit'}, inplace=True)
                        table.drop(index=0, inplace=True)
                        table['total1Unit'] = table['total1Unit'].astype(float)
                        table['total2Unit'] = table['total2Unit'].astype(float)
                        table['total34Unit'] = table['total34Unit'].astype(float)
                        table['total5Unit'] = table['total5Unit'].astype(float)
                        table['value1Unit'] = table['value1Unit'].astype(float)
                        table['value2Unit'] = table['value2Unit'].astype(float)
                        table['value34Unit'] = table['value34Unit'].astype(float)
                        table['value5Unit'] = table['value5Unit'].astype(float)
                        table['totalBuildings10'] = table['total1Unit'] + table['total2Unit'] + table['total34Unit'] + table['total5Unit']
                        table['totalBuildingValue10'] = table['value1Unit'] + table['value2Unit'] + table['value34Unit'] + table['value5Unit']
                        table['GEO_ID'] = "0500000US" + table['State'] + table['County']
                        formatted_columns = ['State', 'County']  # Add more column names as needed
                        for column in formatted_columns:
                            if column == 'State':
                                table[column] = table[column].apply(lambda x: f"{int(x):02}" if x.isdigit() else x)
                            elif column == 'County':
                                table[column] = table[column].apply(lambda x: f"{int(x):03}" if x.isdigit() else x)
                        columns = ['index', 'State', 'County Name','County','totalBuildings10','totalBuildingValue10','GEO_ID']
                        selected = table[columns]
                        csv_file = f"co{str(fourth_digit)}{str(third_digit)}{str(adjusted_last_digits).zfill(2)}{file_type}.csv"
                        selected.to_csv(csv_file, index=False)
                        os.remove(local_file_path)
                    else:
                        print("Failed to download the text file")    
                    if stop_loop:
                        break     
                if adjusted:
                    break
def zillow(driver, unique):
    county_url = 'https://files.zillowstatic.com/research/public_csvs/zhvi/State_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv?t=1690916965'
    driver.get(county_url)
    time.sleep(5)
def climate_hazard(driver, unique):
    url = 'https://hazards.fema.gov/nri/data-resources#csvDownload'
    driver.get(url)
    time.sleep(5)
    download = driver.find_element(By.XPATH, '//*[@id="csvDownload"]/div/div/div[1]/ul/li[1]/a')
    download.click()
    attempts = 0
    max_attempts = 5  # Maximum number of attempts 
    time_break = 10
    while attempts < max_attempts: #trying to do the mechanism 
        files = os.listdir(download_path)
        zipfiles = [file for file in files if file.endswith('.zip')]
        if zipfiles:
            for zip_file in zipfiles:
                zip_file_path = os.path.join(download_path, zip_file)
                with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                    zip_ref.extractall(download_path)
                os.remove(zip_file_path)
            break  # Break out of the retry loop if zip files are found and extracted successfully
        else:
            attempts += 1
            if attempts < max_attempts:
                print(f"No ZIP files found in the download_path. wait {time_break}second for retrying...")
                time.sleep(time_break)
            else:
                print("No ZIP files found after maximum attempts.")
    files = os.listdir(download_path)
    for file in files:
        if 'NRI' in file and not file.endswith('.csv'):
            os.remove(os.path.join(download_path, file))
def Hudgis(driver, unique):
    driver.get(url)
    time.sleep(20)
    download= driver.find_element(By.XPATH, '//*[@id="ember51"]/div/div/div[3]/button')
    download.click()
    time.sleep(10)
    actions = ActionChains(driver)
    x_coord = 346 # x-coordinate
    y_coord = 357  #  Y-coordinate
    actions.move_by_offset(x_coord, y_coord)
    actions.click().perform()
    time.sleep(120)
    
def census_DP02(driver, unique):
    driver.get(url)
    time.sleep(5)
    select = driver.find_element(By.ID, 'toolbar-button-datasets')
    select.click()
    time.sleep(3)
    year_5 = driver.find_element(By.XPATH, '//*[@id="table-header"]/div[2]/div[2]/div/div/div/div[2]/ul/div/li[4]/div[1]/div/div')
    year_5.click()
    wait = WebDriverWait(driver, 10)
    geo = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="toolbar-button-geos"]')))
    geo.click()
    wait = WebDriverWait(driver,10)
    county = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[2]/div/div/div[1]/div/div[2]/div/div/div/section/div[2]/div[2]/div/div[3]')))
    county.click()
    time.sleep(5)
    Clicklist = driver.find_element(By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[2]/div/div/div[1]/div/div[2]/div/div/div/section/ul/li[1]/div/div[1]/div[2]')
    Clicklist.click()
    cancel = driver.find_element(By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[2]/div/div/div[1]/div/div[1]/div/div[1]/div[1]/div/div[4]')
    cancel.click()
    wait = WebDriverWait(driver,15)
    download = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[1]/div[2]/div[2]/div/div/div[4]/div/button')))
    download.click()
    time.sleep(5)
    year_2020= driver.find_element(By.ID, 'Table: DP02 Vintage: 2020')
    year_2020.click()
    time.sleep(1)
    elements_2019= driver.find_elements(By.ID, 'Table: DP02 Vintage: 2019')
    if len(elements_2019) >= 2:
        for element in elements_2019:
            if element == elements_2019[1]:
                element.click()
                break
    time.sleep(1)
    elements_2018= driver.find_elements(By.ID, 'Table: DP02 Vintage: 2018') #there are two same ID 
    if len(elements_2018) >= 2:
        for element in elements_2018:
            if element == elements_2018[1]:
                element.click()
                break
    time.sleep(1)
    elements_2017 = driver.find_elements(By.ID, 'Table: DP02 Vintage: 2017') #there are two same IDs
    if len(elements_2017) >= 2:
        for Element in elements_2017:
            if Element == elements_2017[1]:
                Element.click()
                break
    time.sleep(1)            
    download_csv = driver.find_element(By.XPATH, '//*[@id="app"]/div[3]/div/div/div[2]/div[2]/div[4]/div[2]/button')
    download_csv.click()
    time.sleep(120)
    attempts = 0
    max_attempts = 100  # Maximum number of attempts 
    time_break = 10
    while attempts < max_attempts: #trying to do the mechanism 
        files = os.listdir(download_path)
        zipfiles = [file for file in files if file.endswith('.zip')]
        if zipfiles:
            for zip_file in zipfiles:
                zip_file_path = os.path.join(download_path, zip_file)
                with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                    zip_ref.extractall(download_path)
                os.remove(zip_file_path)
            break  # Break out of the retry loop if zip files are found and extracted successfully
        else:
            attempts += 1
            if attempts < max_attempts:
                print(f"No ZIP files found in the download_path. wait {time_break}second for retrying...")
                time.sleep(time_break)
            else:
                print("No ZIP files found after maximum attempts.")
    time.sleep(10)
    years = range(2017, 2022)
    for year in years:
        csv_file = f'ACSDP5Y{year}.DP02-Data.csv' 
        csv_path = os.path.join(download_path, csv_file)
        table = pd.read_csv(csv_path, header=None, dtype=str)
        table.drop(0, axis=0, inplace=True)
        table.columns = table.iloc[0]
        table = table[1:]
        columns_keyword = []
        for column in table.columns:
            if isinstance(column, str) and ("Geo" in column or "total" in column or "Geo" in column or "fam" in column or "child" in column or "graduate" in column or "imm" in column or "bachelor" in column) and ("Annotation" not in column and "Margin" not in column):
                columns_keyword.append(column)  
        selected_columns = table[columns_keyword]
        selected_columns.to_csv(f'DP02-{year}.csv', index=False)

def census_S2301(driver, unique):
    driver.get(url)
    time.sleep(5)
    select = driver.find_element(By.ID, 'toolbar-button-datasets')
    select.click()
    time.sleep(3)
    year_5 = driver.find_element(By.XPATH, '//*[@id="table-header"]/div[2]/div[2]/div/div/div/div[2]/ul/div/li[2]/div[1]/div/div')
    year_5.click()
    wait = WebDriverWait(driver, 10)
    geo = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="toolbar-button-geos"]')))
    geo.click()
    wait = WebDriverWait(driver,10)
    county = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[2]/div/div/div[1]/div/div[2]/div/div/div/section/div[2]/div[2]/div/div[3]')))
    county.click()
    time.sleep(5)
    Clicklist = driver.find_element(By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[2]/div/div/div[1]/div/div[2]/div/div/div/section/ul/li[1]/div/div[1]/div[2]')
    Clicklist.click()
    cancel = driver.find_element(By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[2]/div/div/div[1]/div/div[1]/div/div[1]/div[1]/div/div[4]')
    cancel.click()
    wait = WebDriverWait(driver,15)
    download = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[1]/div[2]/div[2]/div/div/div[4]/div/button')))
    download.click()
    time.sleep(5)
    year_2020= driver.find_element(By.ID, 'Table: S2301 Vintage: 2020')
    year_2020.click()
    time.sleep(1)
    elements_2019= driver.find_elements(By.ID, 'Table: S2301 Vintage: 2019')
    if len(elements_2019) >= 2:
        for element in elements_2019:
            if element == elements_2019[1]:
                element.click()
                break
    time.sleep(1)
    elements_2018= driver.find_elements(By.ID, 'Table: S2301 Vintage: 2018') #there are two same ID 
    if len(elements_2018) >= 2:
        for element in elements_2018:
            if element == elements_2018[1]:
                element.click()
                break
    time.sleep(1)
    elements_2017 = driver.find_elements(By.ID, 'Table: S2301 Vintage: 2017') #there are two same IDs
    if len(elements_2017) >= 2:
        for Element in elements_2017:
            if Element == elements_2017[1]:
                Element.click()
                break
    time.sleep(1)            
    download_csv = driver.find_element(By.XPATH, '//*[@id="app"]/div[3]/div/div/div[2]/div[2]/div[4]/div[2]/button')
    download_csv.click()
    time.sleep(20)
    attempts = 0
    max_attempts = 100  # Maximum number of attempts 
    time_break = 10
    while attempts < max_attempts: #trying to do the mechanism 
        files = os.listdir(download_path)
        zipfiles = [file for file in files if file.endswith('.zip')]
        if zipfiles:
            for zip_file in zipfiles:
                zip_file_path = os.path.join(download_path, zip_file)
                with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                    zip_ref.extractall(download_path)
                os.remove(zip_file_path)
            break  # Break out of the retry loop if zip files are found and extracted successfully
        else:
            attempts += 1
            if attempts < max_attempts:
                print(f"No ZIP files found in the download_path. wait {time_break}second for retrying...")
                time.sleep(time_break)
            else:
                print("No ZIP files found after maximum attempts.")
    time.sleep(10)
    years = range(2017, 2022)
    for year in years:
        csv_file = f'ACSST5Y{year}.S2301-Data.csv' 
        csv_path = os.path.join(download_path, csv_file)
        table = pd.read_csv(csv_path, header=None, dtype=str)
        table.drop(0, axis=0, inplace=True)
        table.columns = table.iloc[0]
        table = table[1:]
        columns_keyword = []
        for column in table.columns:
            if isinstance(column, str) and ("Geo" in column or "total" in column or "Geo" in column or "Employ" in column or "Unemploy" in column) and ("Annotation" not in column and "Margin" not in column):
                columns_keyword.append(column)  
        selected_columns = table[columns_keyword]
        selected_columns.to_csv(f'S2301-{year}.csv', index=False)
        
def census_B25104(driver, unique):
    driver.get(url)
    time.sleep(5)
    select = driver.find_element(By.ID, 'toolbar-button-datasets')
    select.click()
    time.sleep(3)
    year_5 = driver.find_element(By.XPATH, '//*[@id="table-header"]/div[2]/div[2]/div/div/div/div[2]/ul/div/li[2]/div[1]/div/div')
    year_5.click()
    wait = WebDriverWait(driver, 10)
    geo = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="toolbar-button-geos"]')))
    geo.click()
    wait = WebDriverWait(driver,10)
    county = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[2]/div/div/div[1]/div/div[2]/div/div/div/section/div[2]/div[2]/div/div[3]')))
    county.click()
    time.sleep(5)
    Clicklist = driver.find_element(By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[2]/div/div/div[1]/div/div[2]/div/div/div/section/ul/li[1]/div/div[1]/div[2]')
    Clicklist.click()
    cancel = driver.find_element(By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[2]/div/div/div[1]/div/div[1]/div/div[1]/div[1]/div/div[4]')
    cancel.click()
    wait = WebDriverWait(driver,15)
    download = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[1]/div[2]/div[2]/div/div/div[4]/div/button')))
    download.click()
    time.sleep(5)
    year_2020= driver.find_element(By.ID, 'Table: B25104 Vintage: 2020')
    year_2020.click()
    time.sleep(1)
    elements_2019= driver.find_elements(By.ID, 'Table: B25104 Vintage: 2019')
    if len(elements_2019) >= 2:
        for element in elements_2019:
            if element == elements_2019[1]:
                element.click()
                break
    time.sleep(1)
    elements_2018= driver.find_elements(By.ID, 'Table: B25104 Vintage: 2018') #there are two same ID 
    if len(elements_2018) >= 2:
        for element in elements_2018:
            if element == elements_2018[1]:
                element.click()
                break
    time.sleep(1)
    elements_2017 = driver.find_elements(By.ID, 'Table: B25104 Vintage: 2017') #there are two same IDs
    if len(elements_2017) >= 2:
        for Element in elements_2017:
            if Element == elements_2017[1]:
                Element.click()
                break
    time.sleep(1)            
    download_csv = driver.find_element(By.XPATH, '//*[@id="app"]/div[3]/div/div/div[2]/div[2]/div[4]/div[2]/button')
    download_csv.click()
    time.sleep(20)
    attempts = 0
    max_attempts = 100  # Maximum number of attempts 
    time_break = 10
    while attempts < max_attempts: #trying to do the mechanism 
        files = os.listdir(download_path)
        zipfiles = [file for file in files if file.endswith('.zip')]
        if zipfiles:
            for zip_file in zipfiles:
                zip_file_path = os.path.join(download_path, zip_file)
                with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                    zip_ref.extractall(download_path)
                os.remove(zip_file_path)
            break  # Break out of the retry loop if zip files are found and extracted successfully
        else:
            attempts += 1
            if attempts < max_attempts:
                print(f"No ZIP files found in the download_path. wait {time_break}second for retrying...")
                time.sleep(time_break)
            else:
                print("No ZIP files found after maximum attempts.")
    time.sleep(10)
    years = range(2017, 2022)
    for year in years:
        csv_file = f'ACSDT5Y{year}.B25104-Data.csv' 
        csv_path = os.path.join(download_path, csv_file)
        table = pd.read_csv(csv_path, header=None, dtype=str)
        table.drop(0, axis=0, inplace=True)
        table.columns = table.iloc[0]
        table = table[1:]
        columns_keyword = []
        for column in table.columns:
            if isinstance(column, str) and ("Geo" in column or "Total" in column) and ("Annotation" not in column and "Margin" not in column):
                columns_keyword.append(column)  
        selected_columns = table[columns_keyword]
        selected_columns.to_csv(f'B25104-{year}.csv', index=False)
        os.remove(csv_file)
        
target = {
        "https://data.census.gov/table?q=geographic+mobility&tid=ACSST1Y2021.S0701": census_geographic,
        "https://data.census.gov/table?q=demographics&tid=ACSDP1Y2021.DP05":census_demographics,
        "https://mobility.tamu.edu/umr/report/":tamu_umr_report,
        "https://www.census.gov/econ/bfs/data/county.html":census_data,
        "https://www.census.gov/programs-surveys/cbp/data/datasets.html":census_survey,
        "https://www.cnbc.com/2022/07/13/americas-top-states-for-business-2022-the-full-rankings.html":cnbc_business,
        "https://www.census.gov/data/datasets/time-series/econ/bds/bds-datasets.html":census_time,
        "https://www.realtor.com/research/data/":realtor_research,
        "https://taxfoundation.org/2023-state-business-tax-climate-index/":taxfoundation,
        "https://www.census.gov/construction/bps/index.html":census_construction,
        "https://www.zillow.com/research/data/":zillow,
        "https://hazards.fema.gov/nri/map":climate_hazard,
        "https://hudgis-hud.opendata.arcgis.com/datasets/HUD::location-affordability-index-v-3-1/explore":Hudgis,
        "https://data.census.gov/table?+q=United+States&g=010XX00US&tid=ACSDP5Y2020.DP02":census_DP02,
        "https://data.census.gov/table?q=unemployment&g=010XX00US&tid=ACSST1Y2021.S2301":census_S2301,
        "https://data.census.gov/table?q=monthly+housing+payment&tid=ACSDT1Y2021.B25104":census_B25104
}
            
def check_target(unique):
    target_formats = list(target.keys()) #we save the notation for each target format 
    return any(url.startswith(format) for format in target_formats) #checking whether each url is matched with our target urls under each notation 

for url in unique:
    if check_target(unique):
        interaction_function = target[url] #get the corresponding interaction function
        interaction_function(driver, url)  
driver.quit()    

In [ ]:
download_path = "/Users/frrookie521/Desktop/civil/cee495/"
files = os.listdir(download_path)
zipfiles = [file for file in files if file.endswith('.zip')]
print(zipfiles)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.chrome.options import Options
import os
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException
from IPython.display import display, clear_output
import ipywidgets as widgets
import tkinter as tk
from tkinter import filedialog
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton
import sys
import re
import zipfile  #the import file is zip file
import requests
from bs4 import BeautifulSoup
import datetime #find the current year and date 
PATH = "/Users/frrookie521/Desktop/civil/cee495/cee495/Google\ Chrome\ for\ Testing.app"
download_path = "/Users/frrookie521/Desktop/civil/cee495/"
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
prefs = {
        'download.default_directory': download_path,
        'download.prompt_for_download': False,
        'download.directory_upgrade': True,
        'safebrowsing.enabled': True
         }
chrome_options.add_experimental_option('prefs', prefs)
excel_file = pd.read_excel('VariableSources_2.xlsx', header=None)
driver = webdriver.Chrome(options=chrome_options)
driver.set_window_size(1396, 1000)
urls = excel_file.iloc[1:, 2].tolist() #store urls from third column and second row 
unique = set(url for url in urls if isinstance(url, str)) #extact the unique urls from excel
zipfiles = ['file1.zip', 'file2.zip'] 
county = 'https://www2.census.gov/econ/bps/County/'
driver.get(county)
time.sleep(5)
downloads = driver.find_elements(By.TAG_NAME, 'a')
print(downloads)
def process_txt_content(txt_content, output_csv_file):
    lines = txt_content.split('\n')
    with open(output_csv_file, 'w') as csv_file:
        for line in lines:
            fields = line.strip().split()  # Adjust if needed
            csv_line = ','.join(fields)
            csv_file.write(csv_line + '\n')
            
for download in downloads:
    href = download.get_attribute('href')
    if href:
        files = href.split('/')[-1] #return the last element and save as files
        if 'co' in files and files.endswith('.txt'):
            download.click()
            time.sleep(5)
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'pre')))
            txt_content_element = driver.find_element(By.TAG_NAME, 'pre')
            txt_content = txt_content_element.text
            process_txt_content(txt_content, files.replace('.txt', '.csv'))
            driver.back()
            time.sleep(5)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.chrome.options import Options
import os
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException
from IPython.display import display, clear_output
import ipywidgets as widgets
import tkinter as tk
from tkinter import filedialog
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton
import sys
import re
import zipfile  #the import file is zip file
import requests
from bs4 import BeautifulSoup
import datetime #find the current year and date 
import csv
PATH = "/Users/frrookie521/Desktop/civil/cee495/cee495/Google\ Chrome\ for\ Testing.app"
download_path = "/Users/frrookie521/Desktop/civil/cee495/"
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
prefs = {
        'download.default_directory': download_path,
        'download.prompt_for_download': False,
        'download.directory_upgrade': True,
        'safebrowsing.enabled': True
         }
chrome_options.add_experimental_option('prefs', prefs)
excel_file = pd.read_excel('VariableSources_2.xlsx', header=None)
driver = webdriver.Chrome(options=chrome_options)
driver.set_window_size(1396, 1000)
urls = excel_file.iloc[1:, 2].tolist() #store urls from third column and second row 
unique = set(url for url in urls if isinstance(url, str)) #extact the unique urls from excel
zipfiles = ['file1.zip', 'file2.zip'] 
base = 'https://www2.census.gov/econ/bps/County/'
driver.get(base)
time.sleep(5)
def process_txt_content(txt_content, output_csv_file):
    lines = txt_content.split('\n')
    with open(output_csv_file, 'w') as csv_file:
        for line in lines:
            fields = line.strip().split()  # Adjust if needed
            csv_line = ','.join(fields)
            csv_file.write(csv_line + '\n')
fourth_digit = 1
third_digit = 8   #from year 2018 
stop_loop = False #we define the stop loop for the entire iteration 
generated_filenames = set()
while fourth_digit < 10 and not stop_loop:
    for last_two_digits in range(1, 14):  # The last two digits are changing from 1 to 13
        if fourth_digit == 2 and third_digit == 3 and last_two_digits == 7:
            if file_type == 'c':
                stop_loop = True
                driver.quit()
                break 
        if last_two_digits > 12:
            third_digit += 1
            if third_digit == 10:
                fourth_digit += 1
                third_digit = 0
            adjusted_last_digits = last_two_digits - 12 
            adjusted = True
        else:
            adjusted_last_digits = last_two_digits
            adjusted = False  
        for file_type in ['c', 'y']:
            if fourth_digit == 2 and third_digit == 3 and last_two_digits == 7 and file_type == 'c':
                stop_loop = True
                driver.quit()
                break
            else:
                file_name = f"co{str(fourth_digit)}{str(third_digit)}{str(adjusted_last_digits).zfill(2)}{file_type}.txt"
                if file_name in generated_filenames:
                    continue
                generated_filenames.add(file_name) 
                full_url = base + file_name
                driver.get(full_url)
                driver.refresh()
                time.sleep(2)
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'pre')))
                txt_content_element = driver.find_element(By.TAG_NAME, 'pre')
                txt_content = txt_content_element.text
                process_txt_content(txt_content, file_name.replace('.txt', '.csv'))
                time.sleep(2)
                inputcsv_file = file_name.replace('.txt', '.csv') # we read the csv input based on txt extension label
                table = pd.read_csv(inputcsv_file)
                table['1-unit'] = table['1-unit'].astype(str)
                table['County'][3:] = table['County'][3:] + ' ' + table['Unnamed: 6'][3:] + ' '+ table['1-unit'][3:].astype(str)     
                table['County'] = table['County'].str.replace('nan', '')
                table['Unnamed: 8'] = table['Unnamed: 8'].astype(str)
                table['Unnamed: 9'] = table['Unnamed: 9'].astype(str)
                table.loc[1:, 'Unnamed: 6'] = table['Unnamed: 8'][1:]
                table.loc[1:, '1-unit'] = table['Unnamed: 9'][1:]
                table.loc[1:, 'Unnamed: 9'] = table['2-unit'][1:]
                table['Unnamed: 6'] = table['Unnamed: 6'].str.replace('nan', '0')
                table['1-unit'] = table['1-unit'].str.replace('nan', '0')
                outputcsv_file = inputcsv_file.replace('.csv', '_combined.csv')
                table.to_csv(outputcsv_file, index=False)
            if stop_loop:
                break     
        if adjusted:
            break

In [ ]:
for file_type in ['c', 'y']:
            file_name = f"co{str(fourth_digit)}{str(third_digit)}{str(adjusted_last_digits).zfill(2)}{file_type}.txt"
            full_url = base + file_name
            if fourth_digit == 2 and third_digit == 3 and last_two_digits == 7 and file_type == 'c':
                stop_loop = True 
                driver.quit()
                break
            driver.get(full_url)
            driver.refresh()
            time.sleep(2)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.chrome.options import Options
import os
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException
from IPython.display import display, clear_output
import ipywidgets as widgets
import tkinter as tk
from tkinter import filedialog
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton
import sys
import re
import zipfile  #the import file is zip file
import requests
from bs4 import BeautifulSoup
import datetime #find the current year and date 
import csv
import numpy as np
PATH = "/Users/frrookie521/Desktop/civil/cee495/cee495/Google\ Chrome\ for\ Testing.app"
download_path = "/Users/frrookie521/Desktop/civil/cee495/"
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
prefs = {
        'download.default_directory': download_path,
        'download.prompt_for_download': False,
        'download.directory_upgrade': True,
        'safebrowsing.enabled': True
         }
chrome_options.add_experimental_option('prefs', prefs)
excel_file = pd.read_excel('VariableSources_2.xlsx', header=None)
driver = webdriver.Chrome(options=chrome_options)
driver.set_window_size(1396, 1000)
urls = excel_file.iloc[1:, 2].tolist() #store urls from third column and second row 
unique = set(url for url in urls if isinstance(url, str)) #extact the unique urls from excel
zipfiles = ['file1.zip', 'file2.zip'] 
base = 'https://www2.census.gov/econ/bps/County/'
driver.get(base)
time.sleep(5)
def process_txt_content(txt_content, output_csv_file):
    lines = txt_content.split('\n')
    with open(output_csv_file, 'w') as csv_file:
        for line in lines:
            fields = line.strip().split()  # Adjust if needed
            csv_line = ','.join(fields)
            csv_file.write(csv_line + '\n')
fourth_digit = 1
third_digit = 0   #from year 2018 
stop_loop = False #we define the stop loop for the entire iteration 
generated_filenames = set()
while fourth_digit < 10 and not stop_loop:
    for last_two_digits in range(1, 14):  # The last two digits are changing from 1 to 13
        if fourth_digit == 2 and third_digit == 3 and last_two_digits == 7:
            if file_type == 'c':
                stop_loop = True
                driver.quit()
                break 
        if last_two_digits > 12:
            third_digit += 1
            if third_digit == 10:
                fourth_digit += 1
                third_digit = 0
            adjusted_last_digits = last_two_digits - 12 
            adjusted = True
        else:
            adjusted_last_digits = last_two_digits
            adjusted = False  
        for file_type in ['c', 'y']:
            if fourth_digit == 2 and third_digit == 3 and last_two_digits == 7 and file_type == 'c':
                stop_loop = True
                driver.quit()
                break
            else:
                file_name = f"co{str(fourth_digit)}{str(third_digit)}{str(adjusted_last_digits).zfill(2)}{file_type}.txt"
                if file_name in generated_filenames:
                    continue
                generated_filenames.add(file_name) 
                full_url = base + file_name
                driver.get(full_url)
                driver.refresh()
                time.sleep(2)
                response = requests.get(full_url)
                if response.status_code == 200:
                    content = response.text
                    local_file_path = f"co{str(fourth_digit)}{str(third_digit)}{str(adjusted_last_digits).zfill(2)}{file_type}.txt"
                    with open(local_file_path, "w") as file:
                        file.write(content)
                    table = pd.read_csv(local_file_path, sep=',')
                    table.reset_index(inplace=True)
                    table.drop(columns=['FIPS.1', 'Region', 'Unnamed: 6', 'Unnamed: 9', 'Unnamed: 12', 'Unnamed: 15', 'Unnamed: 17', 'Unnamed: 18', '1-unit rep', 'Unnamed: 20', 'Unnamed: 21', '2-units rep', 'Unnamed: 23', 'Unnamed: 24', '3-4 units rep', 'Unnamed: 26', 'Unnamed: 27', ' 5+units rep'], inplace=True)
                    table.rename(columns={'Survey': 'State', 'FIPS': 'County', 'Division': 'County Name', 'County': 'total1Unit', '1-unit': 'value1Unit', 'Unnamed: 8': 'total2Unit', '2-units': 'value2Unit', 'Unnamed: 11': 'total34Unit', '3-4 units': 'value34Unit', 'Unnamed: 14': 'total5Unit', '5+ units': 'value5Unit'}, inplace=True)
                    table.drop(index=0, inplace=True)
                    table['total1Unit'] = table['total1Unit'].astype(float)
                    table['total2Unit'] = table['total2Unit'].astype(float)
                    table['total34Unit'] = table['total34Unit'].astype(float)
                    table['total5Unit'] = table['total5Unit'].astype(float)
                    table['value1Unit'] = table['value1Unit'].astype(float)
                    table['value2Unit'] = table['value2Unit'].astype(float)
                    table['value34Unit'] = table['value34Unit'].astype(float)
                    table['value5Unit'] = table['value5Unit'].astype(float)
                    table['totalBuildings10'] = table['total1Unit'] + table['total2Unit'] + table['total34Unit'] + table['total5Unit']
                    table['totalBuildingValue10'] = table['value1Unit'] + table['value2Unit'] + table['value34Unit'] + table['value5Unit']
                    table['GEO_ID'] = "0500000US" + table['State'] + table['County']
                    formatted_columns = ['State', 'County']  # Add more column names as needed
                    for column in formatted_columns:
                        if column == 'State':
                            table[column] = table[column].apply(lambda x: f"{int(x):02}" if x.isdigit() else x)
                        elif column == 'County':
                            table[column] = table[column].apply(lambda x: f"{int(x):03}" if x.isdigit() else x)
                    columns = ['index', 'State', 'County Name','County','totalBuildings10','totalBuildingValue10','GEO_ID']
                    selected = table[columns]
                    csv_file = f"co{str(fourth_digit)}{str(third_digit)}{str(adjusted_last_digits).zfill(2)}{file_type}.csv"
                    selected.to_csv(csv_file, index=False)
                    os.remove(local_file_path)
                else:
                    print("Failed to download the text file")    
                if stop_loop:
                    break     
            if adjusted:
                break

In [ ]:
current_year = datetime.datetime.now().year  #get the current year
for year in range(2007, current_year + 1): #we make for loop for each year 
    filtered_columns = [] #we define the empty filtered column 
    if year in [2009, 2013]: #because 2009 and 2013 overall ranking have unique html comparing to other 
        if year == 2009:
            url = "https://www.cnbc.com/id/100000992"
        elif year == 2013: #create additional condition
            url = "https://www.cnbc.com/id/100824779"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('table')
    else: #other years 
        if year >= 2014:
            for month in range(6,8):
                for day in range(10,24):
                    url = f"https://www.cnbc.com/{year}/{month}/{day}/americas-top-states-for-business.html"
                    response = requests.get(url)
                    if response.status_code == 200: #if that url is exisiting, code is 200
                        soup = BeautifulSoup(response.content, 'html.parser')
                        table = soup.find('table')
                        break
                else:
                    continue #creating iteration for checking the html 
                break 
        else:
            url = f"https://www.cnbc.com/top-states-{year}-overall-rankings/" #the url linkfor overall rankings
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')
            table = soup.find('table')
    if table:
        column_names = []
        data_rows = []
        for i, tr in enumerate(table.find_all('tr')):
            if i == 0:  # Header row
                th_tags = tr.find_all(['th', 'td'])
                column_names = [th.text.strip() for th in th_tags if th.text.strip()]
            else:
                row = [td.text.strip() for td in tr.find_all('td')]
                if row:
                    data_rows.append(row)
       
        overall_labels = ["Overall Rank", "Overall","Rank","Overall Ranking"] #multiple labels
        costofbusiness_labels = ["Cost of Business", "Cost of Doing Business"]
        business_columns = ["Business Friendliness","Business Friendly"]
        Infra = ["Infrastructure","Transportation","Infrastructure & Transp.","Infrastructure &amp; Transp."] #does include transportation??
        State = ["State"]
        table_extract = pd.DataFrame(data_rows, columns=column_names)
        table_extract.rename(columns=dict(zip(overall_labels, ['Overall Ranking']*len(overall_labels))), inplace=True)  # Rename into standard form
        table_extract.rename(columns=dict(zip(costofbusiness_labels, ['Cost of Doing Business']*len(costofbusiness_labels))), inplace=True)  # Rename into standard form
        table_extract.rename(columns=dict(zip(Infra, ['Infrastructure']*len(Infra))), inplace=True)  # Rename into standard form
        table_extract.rename(columns=dict(zip(business_columns, ['Business Friendliness']*len(business_columns))), inplace=True)  # Rename into standard form
        for label in overall_labels:
            if label in table_extract.columns:
                filtered_columns.insert(0, label)
                    #rename into standard form 
                break
        for label in costofbusiness_labels:
            if label in table_extract.columns:
                filtered_columns.insert(0, label)
                break
        for label in business_columns:
            if label in table_extract.columns:
                filtered_columns.insert(0, label)
                break
        for label in Infra:
            if label in table_extract.columns:
                filtered_columns.insert(0, label)
                break
        for label in State:
            if label in table_extract.columns:
                filtered_columns.insert(0, label)
                break
        filtered_data = table_extract[filtered_columns]
        filtered_data.to_csv(f'extracted{year}.csv', index=False, header=True, mode='w')
        print(f" {year} downloaded.")

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.chrome.options import Options
import os
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException
from IPython.display import display, clear_output
import ipywidgets as widgets
import tkinter as tk
from tkinter import filedialog
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton
import sys
import re
import zipfile  #the import file is zip file
import requests
from bs4 import BeautifulSoup
import datetime #find the current year and date 
import csv
import numpy as np
table = pd.read_excel('VariableSources_2.xlsx')
table
interested_columns = table.iloc[:, [0, 2]]
interested_columns
grouped = table.groupby('URL')['Variable Name'].apply(list).reset_index()
grouped.columns = ['URL', 'Variable Name']
grouped

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.chrome.options import Options
import os
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException
from IPython.display import display, clear_output
import ipywidgets as widgets
import tkinter as tk
from tkinter import filedialog
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton
import sys
import re
import zipfile  #the import file is zip file
import requests
from bs4 import BeautifulSoup
import datetime #find the current year and date 
import csv
import numpy as np
excelfile = 'complete-data-2021-umr-by-tti.xlsx'
excel_extracted = pd.read_excel(excelfile)
excel_extracted
combined_labels = excel_extracted.iloc[:4].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=0) #combine first 4 row as one label 
excel_extracted = excel_extracted.iloc[4:] #replace first 4 row name
excel_extracted.loc[-1] = combined_labels
excel_extracted.index = excel_extracted.index + 1
excel_extracted.sort_index(inplace=True)
excel_extracted.rename(columns={'Unnamed: 23': 'Annual Hours of Delay'}, inplace=True) #renamed column name 
columns = ['Area Group', 'Urban Area', 'Primary','Population','Year','Arterial Street','Annual Hours of Delay']
selected = excel_extracted[columns]
csv_file_path = 'tmu.csv'
selected.to_csv(csv_file_path, index=False)
os.remove('complete-data-2021-umr-by-tti.xlsx')

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.chrome.options import Options
import os
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException
from IPython.display import display, clear_output
import ipywidgets as widgets
import tkinter as tk
from tkinter import filedialog
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton
import sys
import re
import zipfile  #the import file is zip file
import requests
from bs4 import BeautifulSoup
import datetime #find the current year and date 
PATH = "/Users/frrookie521/Desktop/civil/cee495/cee495/Google\ Chrome\ for\ Testing.app"
download_path = "/Users/frrookie521/Desktop/civil/cee495/"
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
prefs = {
        'download.default_directory': download_path,
        'download.prompt_for_download': False,
        'download.directory_upgrade': True,
        'safebrowsing.enabled': True
         }
chrome_options.add_experimental_option('prefs', prefs)
excel_file = pd.read_excel('VariableSources_2.xlsx', header=None)
driver = webdriver.Chrome(options=chrome_options)
driver.set_window_size(1396, 1000)
urls = excel_file.iloc[1:, 2].tolist() #store urls from third column and second row 
unique = set(url for url in urls if isinstance(url, str)) #extact the unique urls from excel
zipfiles = ['file1.zip', 'file2.zip'] 
url_historical = 'https://econdata.s3-us-west-2.amazonaws.com/Reports/Core/RDC_Inventory_Core_Metrics_County_History.csv'
driver.get(url_historical)
time.sleep(70)
table = pd.read_csv('RDC_Inventory_Core_Metrics_County_History.csv')
table.rename(columns={'month_date_yyyymm': 'year&month'}, inplace=True)
table.rename(columns={'county_name': 'County'}, inplace=True)
table.rename(columns={'county_fips': 'County Federal Information Processing Standard'}, inplace=True)
table.rename(columns={'median_square_feet': 'median listing square feet'}, inplace=True)
columns = ['year&month', 'County', 'County Federal Information Processing Standard','median listing square feet']
selected = table[columns]
csv_file_path = 'realtor.csv'
selected.to_csv(csv_file_path, index=False)
os.remove('RDC_Inventory_Core_Metrics_County_History.csv')

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.chrome.options import Options
import os
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException
from IPython.display import display, clear_output
import ipywidgets as widgets
import tkinter as tk
from tkinter import filedialog
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton
import sys
import re
import zipfile  #the import file is zip file
import requests
from bs4 import BeautifulSoup
import datetime #find the current year and date 
table = pd.read_csv('RDC_Inventory_Core_Metrics_County_History.csv')
table.rename(columns={'month_date_yyyymm': 'year&month'}, inplace=True)
table.rename(columns={'county_name': 'County'}, inplace=True)
table.rename(columns={'county_fips': 'County Federal Information Processing Standard'}, inplace=True)
table.rename(columns={'median_square_feet': 'median listing square feet'}, inplace=True)
columns = ['year&month', 'County', 'County Federal Information Processing Standard','median listing square feet']
selected = table[columns]
csv_file_path = 'realtor.csv'
selected.to_csv(csv_file_path, index=False)
os.remove('RDC_Inventory_Core_Metrics_County_History.csv')

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.chrome.options import Options
import os
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException
from IPython.display import display, clear_output
import ipywidgets as widgets
import tkinter as tk
from tkinter import filedialog
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton
import sys
import re
import zipfile  #the import file is zip file
import requests
from bs4 import BeautifulSoup
import datetime #find the current year and date 
table = pd.read_excel('bfs_county_apps_annual.xlsx', header= None)
table = table.iloc[2:]
selected = table
csv_file_path = 'Business Applications by County.csv'
selected.to_csv(csv_file_path, index=False)
os.remove('bfs_county_apps_annual.xlsx')

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.chrome.options import Options
import os
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException
from IPython.display import display, clear_output
import ipywidgets as widgets
import tkinter as tk
from tkinter import filedialog
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton
import sys
import re
import zipfile  #the import file is zip file
import requests
from bs4 import BeautifulSoup
import datetime #find the current year and date 
def text_to_csv(input_filename, output_filename): #convert txt file to csv 
    with open(input_filename, 'r') as txt: #open the file
        lines = txt.readlines() #read all the lines 
    with open(output_filename, 'w') as csv_file: #set up the csv file 
        for line in lines:
            fields = line.strip().split()  # split the line
            csv_line = ','.join(fields)    # Join fields with commas for CSV
            csv_file.write(csv_line + '\n')
PATH = "/Users/frrookie521/Desktop/civil/cee495/cee495/Google\ Chrome\ for\ Testing.app"
download_path = "/Users/frrookie521/Desktop/civil/cee495/"
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
prefs = {
        'download.default_directory': download_path,
        'download.prompt_for_download': False,
        'download.directory_upgrade': True,
        'safebrowsing.enabled': True
         }
chrome_options.add_experimental_option('prefs', prefs)
excel_file = pd.read_excel('VariableSources_2.xlsx', header=None)
driver = webdriver.Chrome(options=chrome_options)
driver.set_window_size(1396, 1000)
urls = excel_file.iloc[1:, 2].tolist() #store urls from third column and second row 
unique = set(url for url in urls if isinstance(url, str)) #extact the unique urls from excel
zipfiles = ['file1.zip', 'file2.zip'] 
url = 'https://taxfoundation.org/2023-state-business-tax-climate-index/' #the url for 2007 overall raking
response = requests.get(url)  #get the webpage
#require beautifulsoup to extract html documents
Object = BeautifulSoup(response.content, 'html.parser')
table = Object.find('table') #locate the table in the html
column_names = []
first_row = table.find('tr')
for th in first_row.find_all(['th', 'td']):
    column_name = th.text.strip()
    if column_name:
        column_names.append(column_name)
data_rows = []  #we initalize the data row number 
for tr in table.find_all('tr'): #tr represent each number 
    row = [td.text.strip() for td in tr.find_all('td')] #we finds all the row elements within the current row element and create a new list containing the text content of each row element in the row
    if row: #use to check whether there is empty row 
        data_rows.append(row) #if the row is not empty, then we show it in the data_row
table_tax = pd.DataFrame(data_rows, columns=column_names)
table_tax.to_csv('table_tax.csv', index=False)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.chrome.options import Options
import os
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException
from IPython.display import display, clear_output
import ipywidgets as widgets
import tkinter as tk
from tkinter import filedialog
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton
import sys
import re
import zipfile  #the import file is zip file
import requests
from bs4 import BeautifulSoup
import datetime #find the current year and date 
table = pd.read_csv('bds2020_st_cty.csv')
table

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.chrome.options import Options
import os
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException
from IPython.display import display, clear_output
import ipywidgets as widgets
import tkinter as tk
from tkinter import filedialog
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton
import sys
import re
import zipfile  #the import file is zip file
import requests
from bs4 import BeautifulSoup
import datetime #find the current year and date 
download_path = "/Users/frrookie521/Desktop/civil/cee495/"
all_files = os.listdir(download_path)
s0701 = [filename for filename in all_files if 'S0701' in filename] #contain all the lists that have s0701
for filename in s0701:
    if filename.endswith('.csv'):  # if it is csv file 
        file_path = os.path.join(download_path, filename)
        table = pd.read_csv(file_path)
        column_rename_mapping = {
            'United States!!Moved; within same county!!Estimate': 'moved within same county, Estimation',
            'United States!!Moved; within same county!!Margin of Error': 'moved within same county, Margin of Error',
            'United States!!Moved; from different county, same state!!Estimate': 'moved within same state and same county, Estimation',
            'United States!!Moved; from different county, same state!!Margin of Error': 'moved within same state and same county, Margin of Error',
            'United States!!Moved; from different  state!!Estimate': 'moved within different state, Estimate',
            'United States!!Moved; from different  state!!Margin of Error': 'moved within different state, Margin of Error',
            'United States!!Moved; from abroad!!Estimate': 'moved abroad, Estimation',
            'United States!!Moved; from abroad!!Margin of Error': 'moved abroad, Margin of Error',
            'United States!!Total!!Estimate':'Total estimation',
            'United States!!Total!!Margin of Error':'Total Margin of Error'
        }
        
        table.rename(columns=column_rename_mapping, inplace=True)

        table.to_csv(file_path, index=False)
        print(table)


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.chrome.options import Options
import os
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException
from IPython.display import display, clear_output
import ipywidgets as widgets
import tkinter as tk
from tkinter import filedialog
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton
import sys
import re
import zipfile  #the import file is zip file
import requests
from bs4 import BeautifulSoup
import datetime #find the current year and date 
def text_to_csv(input_filename, output_filename): #convert txt file to csv 
    with open(input_filename, 'r') as txt: #open the file
        lines = txt.readlines() #read all the lines 
    with open(output_filename, 'w') as csv_file: #set up the csv file 
        for line in lines:
            fields = line.strip().split()  # split the line
            csv_line = ','.join(fields)    # Join fields with commas for CSV
            csv_file.write(csv_line + '\n')
PATH = "/Users/frrookie521/Desktop/civil/cee495/cee495/Google\ Chrome\ for\ Testing.app"
download_path = "/Users/frrookie521/Desktop/civil/cee495/"
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
prefs = {
        'download.default_directory': download_path,
        'download.prompt_for_download': False,
        'download.directory_upgrade': True,
        'safebrowsing.enabled': True
         }
chrome_options.add_experimental_option('prefs', prefs)
excel_file = pd.read_excel('VariableSources_2.xlsx', header=None)
driver = webdriver.Chrome(options=chrome_options)
driver.set_window_size(1396, 1000)
urls = excel_file.iloc[1:, 2].tolist() #store urls from third column and second row 
unique = set(url for url in urls if isinstance(url, str)) #extact the unique urls from excel
zipfiles = ['file1.zip', 'file2.zip'] 
url = 'https://data.census.gov/table?q=geographic+mobility&tid=ACSST1Y2021.S0701'
driver.get(url)
time.sleep(5)
select = driver.find_element(By.ID, 'toolbar-button-datasets')
select.click()
time.sleep(3)
year_5 = driver.find_element(By.XPATH, '//*[@id="table-header"]/div[2]/div[2]/div/div/div/div[2]/ul/div/li[2]/div[1]/div/div')
year_5.click()
wait = WebDriverWait(driver, 10)
geo = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="toolbar-button-geos"]')))
geo.click()
wait = WebDriverWait(driver,10)
county = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[2]/div/div/div[1]/div/div[2]/div/div/div/section/div[2]/div[2]/div/div[3]')))
county.click()
time.sleep(5)
Clicklist = driver.find_element(By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[2]/div/div/div[1]/div/div[2]/div/div/div/section/ul/li[1]/div/div[1]/div[2]')
Clicklist.click()
cancel = driver.find_element(By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[2]/div/div/div[1]/div/div[1]/div/div[1]/div[1]/div/div[4]')
cancel.click()
wait = WebDriverWait(driver,15)
download = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[1]/div[2]/div[2]/div/div/div[4]/div/button')))
download.click()
time.sleep(3)
year_2020= driver.find_element(By.ID, 'Table: S0701 Vintage: 2020')
year_2020.click()
time.sleep(1)
year_2019= driver.find_element(By.XPATH, '/html/body/div/div/div[3]/div/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div[5]/div/div')
year_2019.click()
time.sleep(1)
elements_2018= driver.find_elements(By.ID, 'Table: S0701 Vintage: 2018') #there are two same ID 
if len(elements_2018) >= 2:
    for element in elements_2018:
        if element == elements_2018[1]:
            element.click()
            break
time.sleep(1)
elements_2017 = driver.find_elements(By.ID, 'Table: S0701 Vintage: 2017') #there are two same IDs
if len(elements_2017) >= 2:
    for Element in elements_2017:
        if Element == elements_2017[1]:
            Element.click()
            break
time.sleep(1)            
download_csv = driver.find_element(By.XPATH, '//*[@id="app"]/div[3]/div/div/div[2]/div[2]/div[4]/div[2]/button')
download_csv.click()
time.sleep(10)
attempts = 0
max_attempts = 5  # Maximum number of attempts 
time_break = 10
while attempts < max_attempts: #trying to do the mechanism 
    files = os.listdir(download_path)
    zipfiles = [file for file in files if file.endswith('.zip')]
    if zipfiles:
        for zip_file in zipfiles:
            zip_file_path = os.path.join(download_path, zip_file)
            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                zip_ref.extractall(download_path)
            os.remove(zip_file_path)
        break  # Break out of the retry loop if zip files are found and extracted successfully
    else:
        attempts += 1
        if attempts < max_attempts:
            print(f"No ZIP files found in the download_path. wait {time_break}second for retrying...")
            time.sleep(time_break)
        else:
            print("No ZIP files found after maximum attempts.")
all_files = os.listdir(download_path)
s0701 = [filename for filename in all_files if 'S0701' in filename] #contain all the lists that have s0701
for filename in s0701:
    if filename.endswith('.csv'):  # if it is csv file 
        file_path = os.path.join(download_path, filename)
        table = pd.read_csv(file_path)
        column_rename_mapping = {
            'United States!!Moved; within same county!!Estimate': 'moved within same county, Estimation',
            'United States!!Moved; within same county!!Margin of Error': 'moved within same county, Margin of Error',
            'United States!!Moved; from different county, same state!!Estimate': 'moved within same state and same county, Estimation',
            'United States!!Moved; from different county, same state!!Margin of Error': 'moved within same state and same county, Margin of Error',
            'United States!!Moved; from different  state!!Estimate': 'moved within different state, Estimate',
            'United States!!Moved; from different  state!!Margin of Error': 'moved within different state, Margin of Error',
            'United States!!Moved; from abroad!!Estimate': 'moved abroad, Estimation',
            'United States!!Moved; from abroad!!Margin of Error': 'moved abroad, Margin of Error',
            'United States!!Total!!Estimate':'Total estimation',
            'United States!!Total!!Margin of Error':'Total Margin of Error'
        }
        
        table.rename(columns=column_rename_mapping, inplace=True)

        table.to_csv(file_path, index=False)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.chrome.options import Options
import os
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException
from IPython.display import display, clear_output
import ipywidgets as widgets
import tkinter as tk
from tkinter import filedialog
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton
import sys
import re
import zipfile  #the import file is zip file
import requests
from bs4 import BeautifulSoup
import datetime #find the current year and date 
years = range(2017, 2021)
for year in years:
    csv_file = f'ACSST5Y{year}.S0701-Data.csv'
    csv_path = os.path.join(download_path, csv_file)
    table = pd.read_csv(csv_path, header=None, dtype=str)
    table.drop(0, axis=0, inplace=True)
    table.columns = table.iloc[0]
    table = table[1:]
    columns_keyword = []
    for column in table.columns:
        if isinstance(column, str) and ("!!Moved" in column or "Geo" in column) and "Annotation" not in column:
            columns_keyword.append(column)  
    selected_columns = table[columns_keyword]
    selected_columns.to_csv(f'{year}.csv', index=False)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.chrome.options import Options
import os
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException
from IPython.display import display, clear_output
import ipywidgets as widgets
import tkinter as tk
from tkinter import filedialog
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton
import sys
import re
import zipfile #extract zip file
import requests
from bs4 import BeautifulSoup
PATH = "/Users/frrookie521/Desktop/civil/cee495/cee495/Google\ Chrome\ for\ Testing.app"
download_path = "/Users/frrookie521/Desktop/civil/cee495/"
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
prefs = {
        'download.default_directory': download_path,
        'download.prompt_for_download': False,
        'download.directory_upgrade': True,
        'safebrowsing.enabled': True
         }
chrome_options.add_experimental_option('prefs', prefs)
excel_file = pd.read_excel('VariableSources_2.xlsx', header=None)
driver = webdriver.Chrome(options=chrome_options)
driver.set_window_size(1396, 1000)
urls = excel_file.iloc[1:, 2].tolist() #store urls from third column and second row 
unique = set(url for url in urls if isinstance(url, str)) #extact the unique urls from excel
zipfiles = ['file1.zip', 'file2.zip'] 
url = 'https://hazards.fema.gov/nri/data-resources#csvDownload'
driver.get(url)
time.sleep(5)
download = driver.find_element(By.XPATH, '//*[@id="csvDownload"]/div/div/div[1]/ul/li[1]/a')
download.click()
attempts = 0
max_attempts = 5  # Maximum number of attempts 
time_break = 10
while attempts < max_attempts: #trying to do the mechanism 
    files = os.listdir(download_path)
    zipfiles = [file for file in files if file.endswith('.zip')]
    if zipfiles:
        for zip_file in zipfiles:
            zip_file_path = os.path.join(download_path, zip_file)
            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                zip_ref.extractall(download_path)
            os.remove(zip_file_path)
        break  # Break out of the retry loop if zip files are found and extracted successfully
    else:
        attempts += 1
        if attempts < max_attempts:
            print(f"No ZIP files found in the download_path. wait {time_break}second for retrying...")
            time.sleep(time_break)
        else:
            print("No ZIP files found after maximum attempts.")
files = os.listdir(download_path)
for file in files:
    if 'NRI' in file and not file.endswith('.csv'):
        os.remove(os.path.join(download_path, file))

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.chrome.options import Options
import os
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException
from IPython.display import display, clear_output
import ipywidgets as widgets
import tkinter as tk
from tkinter import filedialog
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton
import sys
import re
import zipfile #extract zip file
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
PATH = "/Users/frrookie521/Desktop/civil/cee495/cee495/Google\ Chrome\ for\ Testing.app"
download_path = "/Users/frrookie521/Desktop/civil/cee495/"
chrome_options = Options()
chrome_options.page_load_strategy = 'eager'
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
prefs = {
        'download.default_directory': download_path,
        'download.prompt_for_download': False,
        'download.directory_upgrade': True,
        'safebrowsing.enabled': True
         }
chrome_options.add_experimental_option('prefs', prefs)
desired_capabilities = DesiredCapabilities.CHROME.copy()
desired_capabilities['goog:loggingPrefs'] = {'browser': 'ALL'}
excel_file = pd.read_excel('VariableSources_2.xlsx', header=None)
driver = webdriver.Chrome(
    options=chrome_options
)
driver.set_window_size(1396, 1000)
urls = excel_file.iloc[1:, 2].tolist() #store urls from third column and second row 
unique = set(url for url in urls if isinstance(url, str)) #extact the unique urls from excel
zipfiles = ['file1.zip', 'file2.zip'] 
url = 'https://hudgis-hud.opendata.arcgis.com/datasets/HUD::location-affordability-index-v-3-1/explore'
driver.get(url)
time.sleep(20)
download = driver.find_element(By.CSS_SELECTOR, "//*[@id='ember51']/div/div/div[3]/button")
download.click()
download_button = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="ember51"]/div/div/div[1]/div/div/div[3]/hub-download-card')))
download_button.click()
time.sleep(60)

# DP02

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.chrome.options import Options
import os
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException
from IPython.display import display, clear_output
import ipywidgets as widgets
import tkinter as tk
from tkinter import filedialog
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton
import sys
import re
import zipfile #extract zip file
import requests
from bs4 import BeautifulSoup
PATH = "/Users/frrookie521/Desktop/civil/cee495/cee495/Google\ Chrome\ for\ Testing.app"
download_path = "/Users/frrookie521/Desktop/civil/cee495/"
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
prefs = {
        'download.default_directory': download_path,
        'download.prompt_for_download': False,
        'download.directory_upgrade': True,
        'safebrowsing.enabled': True
         }
chrome_options.add_experimental_option('prefs', prefs)
excel_file = pd.read_excel('VariableSources_2.xlsx', header=None)
driver = webdriver.Chrome(options=chrome_options)
driver.set_window_size(1396, 1000)
urls = excel_file.iloc[1:, 2].tolist() #store urls from third column and second row 
unique = set(url for url in urls if isinstance(url, str)) #extact the unique urls from excel
zipfiles = ['file1.zip', 'file2.zip'] 
url = 'https://data.census.gov/table?+q=United+States&g=010XX00US&tid=ACSDP5Y2020.DP02'
driver.get(url)
time.sleep(5)
select = driver.find_element(By.ID, 'toolbar-button-datasets')
select.click()
time.sleep(3)
year_5 = driver.find_element(By.XPATH, '//*[@id="table-header"]/div[2]/div[2]/div/div/div/div[2]/ul/div/li[4]/div[1]/div/div')
year_5.click()
wait = WebDriverWait(driver, 10)
geo = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="toolbar-button-geos"]')))
geo.click()
wait = WebDriverWait(driver,10)
county = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[2]/div/div/div[1]/div/div[2]/div/div/div/section/div[2]/div[2]/div/div[3]')))
county.click()
time.sleep(5)
Clicklist = driver.find_element(By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[2]/div/div/div[1]/div/div[2]/div/div/div/section/ul/li[1]/div/div[1]/div[2]')
Clicklist.click()
cancel = driver.find_element(By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[2]/div/div/div[1]/div/div[1]/div/div[1]/div[1]/div/div[4]')
cancel.click()
wait = WebDriverWait(driver,15)
download = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[1]/div[2]/div[2]/div/div/div[4]/div/button')))
download.click()
time.sleep(5)
year_2020= driver.find_element(By.ID, 'Table: DP02 Vintage: 2020')
year_2020.click()
time.sleep(1)
elements_2019= driver.find_elements(By.ID, 'Table: DP02 Vintage: 2019')
if len(elements_2019) >= 2:
    for element in elements_2019:
        if element == elements_2019[1]:
            element.click()
            break
time.sleep(1)
elements_2018= driver.find_elements(By.ID, 'Table: DP02 Vintage: 2018') #there are two same ID 
if len(elements_2018) >= 2:
    for element in elements_2018:
        if element == elements_2018[1]:
            element.click()
            break
time.sleep(1)
elements_2017 = driver.find_elements(By.ID, 'Table: DP02 Vintage: 2017') #there are two same IDs
if len(elements_2017) >= 2:
    for Element in elements_2017:
        if Element == elements_2017[1]:
            Element.click()
            break
time.sleep(1)            
download_csv = driver.find_element(By.XPATH, '//*[@id="app"]/div[3]/div/div/div[2]/div[2]/div[4]/div[2]/button')
download_csv.click()
time.sleep(120)
attempts = 0
max_attempts = 100  # Maximum number of attempts 
time_break = 10
while attempts < max_attempts: #trying to do the mechanism 
    files = os.listdir(download_path)
    zipfiles = [file for file in files if file.endswith('.zip')]
    if zipfiles:
        for zip_file in zipfiles:
            zip_file_path = os.path.join(download_path, zip_file)
            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                zip_ref.extractall(download_path)
            os.remove(zip_file_path)
        break  # Break out of the retry loop if zip files are found and extracted successfully
    else:
        attempts += 1
        if attempts < max_attempts:
            print(f"No ZIP files found in the download_path. wait {time_break}second for retrying...")
            time.sleep(time_break)
        else:
            print("No ZIP files found after maximum attempts.")
time.sleep(10)
years = range(2017, 2022)
for year in years:
    csv_file = f'ACSDP5Y{year}.DP02-Data.csv' 
    csv_path = os.path.join(download_path, csv_file)
    table = pd.read_csv(csv_path, header=None, dtype=str)
    table.drop(0, axis=0, inplace=True)
    table.columns = table.iloc[0]
    table = table[1:]
    columns_keyword = []
    for column in table.columns:
        if isinstance(column, str) and ("Geo" in column or "total" in column or "Geo" in column or "fam" in column or "child" in column or "graduate" in column or "imm" in column or "bachelor" in column) and ("Annotation" not in column and "Margin" not in column):
            columns_keyword.append(column)  
    selected_columns = table[columns_keyword]
    selected_columns.to_csv(f'DP02-{year}.csv', index=False)

# S2301

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.chrome.options import Options
import os
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException
from IPython.display import display, clear_output
import ipywidgets as widgets
import tkinter as tk
from tkinter import filedialog
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton
import sys
import re
import zipfile #extract zip file
import requests
from bs4 import BeautifulSoup
PATH = "/Users/frrookie521/Desktop/civil/cee495/cee495/Google\ Chrome\ for\ Testing.app"
download_path = "/Users/frrookie521/Desktop/civil/cee495/"
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
prefs = {
        'download.default_directory': download_path,
        'download.prompt_for_download': False,
        'download.directory_upgrade': True,
        'safebrowsing.enabled': True
         }
chrome_options.add_experimental_option('prefs', prefs)
excel_file = pd.read_excel('VariableSources_2.xlsx', header=None)
driver = webdriver.Chrome(options=chrome_options)
driver.set_window_size(1396, 1000)
urls = excel_file.iloc[1:, 2].tolist() #store urls from third column and second row 
unique = set(url for url in urls if isinstance(url, str)) #extact the unique urls from excel
zipfiles = ['file1.zip', 'file2.zip'] 
url = 'https://data.census.gov/table?q=unemployment&g=010XX00US&tid=ACSST1Y2021.S2301'
driver.get(url)
time.sleep(5)
select = driver.find_element(By.ID, 'toolbar-button-datasets')
select.click()
time.sleep(3)
year_5 = driver.find_element(By.XPATH, '//*[@id="table-header"]/div[2]/div[2]/div/div/div/div[2]/ul/div/li[2]/div[1]/div/div')
year_5.click()
wait = WebDriverWait(driver, 10)
geo = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="toolbar-button-geos"]')))
geo.click()
wait = WebDriverWait(driver,10)
county = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[2]/div/div/div[1]/div/div[2]/div/div/div/section/div[2]/div[2]/div/div[3]')))
county.click()
time.sleep(5)
Clicklist = driver.find_element(By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[2]/div/div/div[1]/div/div[2]/div/div/div/section/ul/li[1]/div/div[1]/div[2]')
Clicklist.click()
cancel = driver.find_element(By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[2]/div/div/div[1]/div/div[1]/div/div[1]/div[1]/div/div[4]')
cancel.click()
wait = WebDriverWait(driver,15)
download = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[1]/div[2]/div[2]/div/div/div[4]/div/button')))
download.click()
time.sleep(5)
year_2020= driver.find_element(By.ID, 'Table: S2301 Vintage: 2020')
year_2020.click()
time.sleep(1)
elements_2019= driver.find_elements(By.ID, 'Table: S2301 Vintage: 2019')
if len(elements_2019) >= 2:
    for element in elements_2019:
        if element == elements_2019[1]:
            element.click()
            break
time.sleep(1)
elements_2018= driver.find_elements(By.ID, 'Table: S2301 Vintage: 2018') #there are two same ID 
if len(elements_2018) >= 2:
    for element in elements_2018:
        if element == elements_2018[1]:
            element.click()
            break
time.sleep(1)
elements_2017 = driver.find_elements(By.ID, 'Table: S2301 Vintage: 2017') #there are two same IDs
if len(elements_2017) >= 2:
    for Element in elements_2017:
        if Element == elements_2017[1]:
            Element.click()
            break
time.sleep(1)            
download_csv = driver.find_element(By.XPATH, '//*[@id="app"]/div[3]/div/div/div[2]/div[2]/div[4]/div[2]/button')
download_csv.click()
time.sleep(20)
attempts = 0
max_attempts = 100  # Maximum number of attempts 
time_break = 10
while attempts < max_attempts: #trying to do the mechanism 
    files = os.listdir(download_path)
    zipfiles = [file for file in files if file.endswith('.zip')]
    if zipfiles:
        for zip_file in zipfiles:
            zip_file_path = os.path.join(download_path, zip_file)
            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                zip_ref.extractall(download_path)
            os.remove(zip_file_path)
        break  # Break out of the retry loop if zip files are found and extracted successfully
    else:
        attempts += 1
        if attempts < max_attempts:
            print(f"No ZIP files found in the download_path. wait {time_break}second for retrying...")
            time.sleep(time_break)
        else:
            print("No ZIP files found after maximum attempts.")
time.sleep(10)
years = range(2017, 2022)
for year in years:
    csv_file = f'ACSST5Y{year}.S2301-Data.csv' 
    csv_path = os.path.join(download_path, csv_file)
    table = pd.read_csv(csv_path, header=None, dtype=str)
    table.drop(0, axis=0, inplace=True)
    table.columns = table.iloc[0]
    table = table[1:]
    columns_keyword = []
    for column in table.columns:
        if isinstance(column, str) and ("Geo" in column or "total" in column or "Geo" in column or "Employ" in column or "Unemploy" in column) and ("Annotation" not in column and "Margin" not in column):
            columns_keyword.append(column)  
    selected_columns = table[columns_keyword]
    selected_columns.to_csv(f'S2301-{year}.csv', index=False)

# B25104

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.chrome.options import Options
import os
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException
from IPython.display import display, clear_output
import ipywidgets as widgets
import tkinter as tk
from tkinter import filedialog
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton
import sys
import re
import zipfile #extract zip file
import requests
from bs4 import BeautifulSoup
PATH = "/Users/frrookie521/Desktop/civil/cee495/cee495/Google\ Chrome\ for\ Testing.app"
download_path = "/Users/frrookie521/Desktop/civil/cee495/"
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
prefs = {
        'download.default_directory': download_path,
        'download.prompt_for_download': False,
        'download.directory_upgrade': True,
        'safebrowsing.enabled': True
         }
chrome_options.add_experimental_option('prefs', prefs)
excel_file = pd.read_excel('VariableSources_2.xlsx', header=None)
driver = webdriver.Chrome(options=chrome_options)
driver.set_window_size(1396, 1000)
urls = excel_file.iloc[1:, 2].tolist() #store urls from third column and second row 
unique = set(url for url in urls if isinstance(url, str)) #extact the unique urls from excel
zipfiles = ['file1.zip', 'file2.zip'] 
url = 'https://data.census.gov/table?q=monthly+housing+payment&tid=ACSDT1Y2021.B25104'
driver.get(url)
time.sleep(5)
select = driver.find_element(By.ID, 'toolbar-button-datasets')
select.click()
time.sleep(3)
year_5 = driver.find_element(By.XPATH, '//*[@id="table-header"]/div[2]/div[2]/div/div/div/div[2]/ul/div/li[2]/div[1]/div/div')
year_5.click()
wait = WebDriverWait(driver, 10)
geo = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="toolbar-button-geos"]')))
geo.click()
wait = WebDriverWait(driver,10)
county = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[2]/div/div/div[1]/div/div[2]/div/div/div/section/div[2]/div[2]/div/div[3]')))
county.click()
time.sleep(5)
Clicklist = driver.find_element(By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[2]/div/div/div[1]/div/div[2]/div/div/div/section/ul/li[1]/div/div[1]/div[2]')
Clicklist.click()
cancel = driver.find_element(By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[2]/div/div/div[1]/div/div[1]/div/div[1]/div[1]/div/div[4]')
cancel.click()
wait = WebDriverWait(driver,15)
download = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[1]/div[2]/div[2]/div/div/div[4]/div/button')))
download.click()
time.sleep(5)
year_2020= driver.find_element(By.ID, 'Table: B25104 Vintage: 2020')
year_2020.click()
time.sleep(1)
elements_2019= driver.find_elements(By.ID, 'Table: B25104 Vintage: 2019')
if len(elements_2019) >= 2:
    for element in elements_2019:
        if element == elements_2019[1]:
            element.click()
            break
time.sleep(1)
elements_2018= driver.find_elements(By.ID, 'Table: B25104 Vintage: 2018') #there are two same ID 
if len(elements_2018) >= 2:
    for element in elements_2018:
        if element == elements_2018[1]:
            element.click()
            break
time.sleep(1)
elements_2017 = driver.find_elements(By.ID, 'Table: B25104 Vintage: 2017') #there are two same IDs
if len(elements_2017) >= 2:
    for Element in elements_2017:
        if Element == elements_2017[1]:
            Element.click()
            break
time.sleep(1)            
download_csv = driver.find_element(By.XPATH, '//*[@id="app"]/div[3]/div/div/div[2]/div[2]/div[4]/div[2]/button')
download_csv.click()
time.sleep(20)
attempts = 0
max_attempts = 100  # Maximum number of attempts 
time_break = 10
while attempts < max_attempts: #trying to do the mechanism 
    files = os.listdir(download_path)
    zipfiles = [file for file in files if file.endswith('.zip')]
    if zipfiles:
        for zip_file in zipfiles:
            zip_file_path = os.path.join(download_path, zip_file)
            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                zip_ref.extractall(download_path)
            os.remove(zip_file_path)
        break  # Break out of the retry loop if zip files are found and extracted successfully
    else:
        attempts += 1
        if attempts < max_attempts:
            print(f"No ZIP files found in the download_path. wait {time_break}second for retrying...")
            time.sleep(time_break)
        else:
            print("No ZIP files found after maximum attempts.")
time.sleep(10)
years = range(2017, 2022)
for year in years:
    csv_file = f'ACSDT5Y{year}.B25104-Data.csv' 
    csv_path = os.path.join(download_path, csv_file)
    table = pd.read_csv(csv_path, header=None, dtype=str)
    table.drop(0, axis=0, inplace=True)
    table.columns = table.iloc[0]
    table = table[1:]
    columns_keyword = []
    for column in table.columns:
        if isinstance(column, str) and ("Geo" in column or "Total" in column) and ("Annotation" not in column and "Margin" not in column):
            columns_keyword.append(column)  
    selected_columns = table[columns_keyword]
    selected_columns.to_csv(f'B25104-{year}.csv', index=False)
    os.remove(csv_file)

# S0802

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.chrome.options import Options
import os
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException
from IPython.display import display, clear_output
import ipywidgets as widgets
import tkinter as tk
from tkinter import filedialog
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton
import sys
import re
import zipfile #extract zip file
import requests
from bs4 import BeautifulSoup
PATH = "/Users/frrookie521/Desktop/civil/cee495/cee495/Google\ Chrome\ for\ Testing.app"
download_path = "/Users/frrookie521/Desktop/civil/cee495/"
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
prefs = {
        'download.default_directory': download_path,
        'download.prompt_for_download': False,
        'download.directory_upgrade': True,
        'safebrowsing.enabled': True
         }
chrome_options.add_experimental_option('prefs', prefs)
excel_file = pd.read_excel('VariableSources_2.xlsx', header=None)
driver = webdriver.Chrome(options=chrome_options)
driver.set_window_size(1396, 1000)
urls = excel_file.iloc[1:, 2].tolist() #store urls from third column and second row 
unique = set(url for url in urls if isinstance(url, str)) #extact the unique urls from excel
zipfiles = ['file1.zip', 'file2.zip'] 
url = 'https://data.census.gov/table?q=commute&g=010XX00US&tid=ACSST1Y2021.S0802'
driver.get(url)
time.sleep(5)
select = driver.find_element(By.ID, 'toolbar-button-datasets')
select.click()
time.sleep(3)
year_5 = driver.find_element(By.XPATH, '//*[@id="table-header"]/div[2]/div[2]/div/div/div/div[2]/ul/div/li[2]/div[1]/div/div')
year_5.click()
wait = WebDriverWait(driver, 10)
geo = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="toolbar-button-geos"]')))
geo.click()
wait = WebDriverWait(driver,10)
county = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[2]/div/div/div[1]/div/div[2]/div/div/div/section/div[2]/div[2]/div/div[3]')))
county.click()
time.sleep(5)
Clicklist = driver.find_element(By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[2]/div/div/div[1]/div/div[2]/div/div/div/section/ul/li[1]/div/div[1]/div[2]')
Clicklist.click()
cancel = driver.find_element(By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[2]/div/div/div[1]/div/div[1]/div/div[1]/div[1]/div/div[4]')
cancel.click()
wait = WebDriverWait(driver,15)
download = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[1]/div[2]/div[2]/div/div/div[4]/div/button')))
download.click()
time.sleep(5)
year_2020= driver.find_element(By.ID, 'Table: S0802 Vintage: 2020')
year_2020.click()
time.sleep(1)
elements_2019= driver.find_elements(By.ID, 'Table: S0802 Vintage: 2019')
if len(elements_2019) >= 2:
    for element in elements_2019:
        if element == elements_2019[1]:
            element.click()
            break
time.sleep(1)
elements_2018= driver.find_elements(By.ID, 'Table: S0802 Vintage: 2018') #there are two same ID 
if len(elements_2018) >= 2:
    for element in elements_2018:
        if element == elements_2018[1]:
            element.click()
            break
time.sleep(1)
elements_2017 = driver.find_elements(By.ID, 'Table: S0802 Vintage: 2017') #there are two same IDs
if len(elements_2017) >= 2:
    for Element in elements_2017:
        if Element == elements_2017[1]:
            Element.click()
            break
time.sleep(1)            
download_csv = driver.find_element(By.XPATH, '//*[@id="app"]/div[3]/div/div/div[2]/div[2]/div[4]/div[2]/button')
download_csv.click()
time.sleep(10)
attempts = 0
max_attempts = 100  # Maximum number of attempts 
time_break = 10
while attempts < max_attempts: #trying to do the mechanism 
    files = os.listdir(download_path)
    zipfiles = [file for file in files if file.endswith('.zip')]
    if zipfiles:
        for zip_file in zipfiles:
            zip_file_path = os.path.join(download_path, zip_file)
            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                zip_ref.extractall(download_path)
            os.remove(zip_file_path)
        break  # Break out of the retry loop if zip files are found and extracted successfully
    else:
        attempts += 1
        if attempts < max_attempts:
            time.sleep(time_break)
        else:
            print("No ZIP files found after maximum attempts.")
years = range(2017, 2022)
for year in years:
    csv_file = f'ACSST5Y{year}.S0802-Data.csv' 
    csv_path = os.path.join(download_path, csv_file)
    table = pd.read_csv(csv_path, header=None, dtype=str)
    table.drop(0, axis=0, inplace=True)
    table.columns = table.iloc[0]
    table = table[1:]
    columns_keyword = []
    for column in table.columns:
        if isinstance(column, str) and ("Geo" in column or "TRAVEL TIME " in column or "Median earnings" in column) and ("Annotation" not in column and "Margin" not in column):
            columns_keyword.append(column)  
    selected_columns = table[columns_keyword]
    selected_columns.to_csv(f'S0802-{year}.csv', index=False)

# B14004

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.chrome.options import Options
import os
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException
from IPython.display import display, clear_output
import ipywidgets as widgets
import tkinter as tk
from tkinter import filedialog
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton
import sys
import re
import zipfile #extract zip file
import requests
from bs4 import BeautifulSoup
PATH = "/Users/frrookie521/Desktop/civil/cee495/cee495/Google\ Chrome\ for\ Testing.app"
download_path = "/Users/frrookie521/Desktop/civil/cee495/"
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
prefs = {
        'download.default_directory': download_path,
        'download.prompt_for_download': False,
        'download.directory_upgrade': True,
        'safebrowsing.enabled': True
         }
chrome_options.add_experimental_option('prefs', prefs)
excel_file = pd.read_excel('VariableSources_2.xlsx', header=None)
driver = webdriver.Chrome(options=chrome_options)
driver.set_window_size(1396, 1000)
urls = excel_file.iloc[1:, 2].tolist() #store urls from third column and second row 
unique = set(url for url in urls if isinstance(url, str)) #extact the unique urls from excel
zipfiles = ['file1.zip', 'file2.zip'] 
url = 'https://data.census.gov/table?q=B14004:+SEX+BY+COLLEGE+OR+GRADUATE+SCHOOL+ENROLLMENT+BY+TYPE+OF+SCHOOL+BY+AGE+FOR+THE+POPULATION+15+YEARS+AND+OVER&tid=ACSDT1Y2021.B14004'
driver.get(url)
time.sleep(5)
select = driver.find_element(By.ID, 'toolbar-button-datasets')
select.click()
time.sleep(3)
year_5 = driver.find_element(By.XPATH, '//*[@id="table-header"]/div[2]/div[2]/div/div/div/div[2]/ul/div/li[2]/div[1]')
year_5.click()
wait = WebDriverWait(driver, 10)
geo = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="toolbar-button-geos"]')))
geo.click()
wait = WebDriverWait(driver,10)
county = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[2]/div/div/div[1]/div/div[2]/div/div/div/section/div[2]/div[2]/div/div[3]')))
county.click()
time.sleep(5)
Clicklist = driver.find_element(By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[2]/div/div/div[1]/div/div[2]/div/div/div/section/ul/li[1]/div/div[1]/div[2]')
Clicklist.click()
cancel = driver.find_element(By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[2]/div/div/div[1]/div/div[1]/div/div[1]/div[1]/div/div[4]')
cancel.click()
wait = WebDriverWait(driver,15)
download = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[4]/main/div[1]/div[2]/div[2]/div/div/div[4]/div/button')))
download.click()
time.sleep(5)
year_2020= driver.find_element(By.ID, 'Table: B14004 Vintage: 2020')
year_2020.click()
time.sleep(1)
elements_2019= driver.find_elements(By.ID, 'Table: B14004 Vintage: 2019')
if len(elements_2019) >= 2:
    for element in elements_2019:
        if element == elements_2019[1]:
            element.click()
            break
time.sleep(1)
elements_2018= driver.find_elements(By.ID, 'Table: B14004 Vintage: 2018') #there are two same ID 
if len(elements_2018) >= 2:
    for element in elements_2018:
        if element == elements_2018[1]:
            element.click()
            break
time.sleep(1)
elements_2017 = driver.find_elements(By.ID, 'Table: B14004 Vintage: 2017') #there are two same IDs
if len(elements_2017) >= 2:
    for Element in elements_2017:
        if Element == elements_2017[1]:
            Element.click()
            break
time.sleep(1)            
download_csv = driver.find_element(By.XPATH, '//*[@id="app"]/div[3]/div/div/div[2]/div[2]/div[4]/div[2]/button')
download_csv.click()
time.sleep(120)
time.sleep(10)
attempts = 0
max_attempts = 100  # Maximum number of attempts 
time_break = 10
while attempts < max_attempts: #trying to do the mechanism 
    files = os.listdir(download_path)
    zipfiles = [file for file in files if file.endswith('.zip')]
    if zipfiles:
        for zip_file in zipfiles:
            zip_file_path = os.path.join(download_path, zip_file)
            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                zip_ref.extractall(download_path)
            os.remove(zip_file_path)
        break  # Break out of the retry loop if zip files are found and extracted successfully
    else:
        attempts += 1
        if attempts < max_attempts:
            time.sleep(time_break)
        else:
            print("No ZIP files found after maximum attempts.")

# libraray

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.chrome.options import Options
import os
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException
from IPython.display import display, clear_output
import ipywidgets as widgets
import tkinter as tk
from tkinter import filedialog
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton
import sys
import re
import zipfile #extract zip file
import requests
from bs4 import BeautifulSoup
import shutil
PATH = "/Users/frrookie521/Desktop/civil/cee495/cee495/Google\ Chrome\ for\ Testing.app"
download_path = "/Users/frrookie521/Desktop/civil/cee495/"
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
prefs = {
        'download.default_directory': download_path,
        'download.prompt_for_download': False,
        'download.directory_upgrade': True,
        'safebrowsing.enabled': True
         }
chrome_options.add_experimental_option('prefs', prefs)
excel_file = pd.read_excel('VariableSources_2.xlsx', header=None)
driver = webdriver.Chrome(options=chrome_options)
driver.set_window_size(1396, 1000)
urls = excel_file.iloc[1:, 2].tolist() #store urls from third column and second row 
unique = set(url for url in urls if isinstance(url, str)) #extact the unique urls from excel
zipfiles = ['file1.zip', 'file2.zip'] 
url = 'https://www.imls.gov/research-evaluation/data-collection/public-libraries-survey'
driver.get(url)
time.sleep(5)
driver.execute_script("window.scrollBy(0, 1000);") 
time.sleep(3)
year_2021 = driver.find_element(By.ID, 'fy-2021')
year_2021.click()
time.sleep(1)
download = driver.find_element(By.XPATH, '//*[@id="ui-id-1"]/ul/li[1]/a[1]')
download.click()
time.sleep(5)
attempts = 0
max_attempts = 100  # Maximum number of attempts 
time_break = 10
while attempts < max_attempts: #trying to do the mechanism 
    files = os.listdir(download_path)
    zipfiles = [file for file in files if file.endswith('.zip')]
    if zipfiles:
        for zip_file in zipfiles:
            zip_file_path = os.path.join(download_path, zip_file)
            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                zip_ref.extractall(download_path)
            os.remove(zip_file_path)
        break  # Break out of the retry loop if zip files are found and extracted successfully
    else:
        attempts += 1
        if attempts < max_attempts:
            time.sleep(time_break)
        else:
            print("No ZIP files found after maximum attempts.")
time.sleep(1)
extracted_folder = "/Users/frrookie521/Desktop/civil/cee495/PLS_FY2021 PUD_CSV/"
download_path = "/Users/frrookie521/Desktop/civil/cee495/"
csv_files = ["PLS_FY21_AE_pud21i.csv", "pls_fy21_outlet_pud21i.csv"]
for csv_filename in csv_files:
    extracted_file_path = os.path.join(extracted_folder, csv_filename)
    download_file_path = os.path.join(download_path, csv_filename)
    try:
        shutil.move(extracted_file_path, download_file_path)
    except Exception as e:
        print(f"An error extracting the file '{csv_filename}'")   
shutil.rmtree(extracted_folder)

# Interstate

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.chrome.options import Options
import os
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException
from IPython.display import display, clear_output
import ipywidgets as widgets
import tkinter as tk
from tkinter import filedialog
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton
import sys
import re
import zipfile #extract zip file
import requests
from bs4 import BeautifulSoup
import shutil
PATH = "/Users/frrookie521/Desktop/civil/cee495/cee495/Google\ Chrome\ for\ Testing.app"
download_path = "/Users/frrookie521/Desktop/civil/cee495/"
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
prefs = {
        'download.default_directory': download_path,
        'download.prompt_for_download': False,
        'download.directory_upgrade': True,
        'safebrowsing.enabled': True
         }
chrome_options.add_experimental_option('prefs', prefs)
excel_file = pd.read_excel('VariableSources_2.xlsx', header=None)
driver = webdriver.Chrome(options=chrome_options)
driver.set_window_size(1396, 1000)
urls = excel_file.iloc[1:, 2].tolist() #store urls from third column and second row 
unique = set(url for url in urls if isinstance(url, str)) #extact the unique urls from excel
zipfiles = ['file1.zip', 'file2.zip'] 
years = range(2015, 2022)
download_xpath_years = {
    2015: '//*[@id="fullpage"]/table[3]/tbody/tr[4]/td[3]/a',
    2016: '//*[@id="fullpage"]/table[4]/tbody/tr[4]/td[3]/a',
    2017: '//*[@id="fullpage"]/table[3]/tbody/tr[4]/td[3]/a',
    2018: '//*[@id="fullpage"]/table[3]/tbody/tr[4]/td[3]/a',
    2019: '//*[@id="fullpage"]/table[3]/tbody/tr[4]/td[3]/a',
    2020: '//*[@id="fullpage"]/table[3]/tbody/tr[4]/td[3]/a',
    2021: '//*[@id="fullpage"]/table[3]/tbody/tr[5]/td[3]/a'
}

for year in years:
    html = f'https://www.fhwa.dot.gov/policyinformation/statistics/{year}/index.cfm#sec4' 
    driver.get(html)
    time.sleep(1)
    download_xpath = download_xpath_years.get(year) #select the correct xpath based on year     
    if download_xpath:
        download = driver.find_element(By.XPATH, download_xpath)
        download.click()
        time.sleep(2)
        old_excel = 'vmt422c.xls'
        if year == 2021:
            old_excel = 'hm220.xls'
        new_excel = f"{year}_interstate.xls"
        os.rename(old_excel, new_excel)
    else:
        print(f"XPath for {year} not found.")
table = pd.read_excel('2021_interstate.xls')
rows_to_search = [6, 7]
filtered_columns = []
keywords = ['Interstate', 'Total', 'Year']
for row in rows_to_search:
        filtered_columns += [col for col in table.columns if any(keyword.lower() in str(table.iloc[row][col]).lower() for keyword in keywords)]
new_table = table[filtered_columns]
new_table.reset_index(drop=True, inplace=True)
new_table = new_table.iloc[6:]
new_table = new_table.drop(8)
new_table.reset_index(drop=True, inplace=True)
new_table.iat[1, 0] = 'YEAR'
new_table.iat[1, 1] = 'Total rural and urban (Miles)'
new_table.iat[1, 2] = 'Rural interstate (Miles)'
new_table.iat[1, 3] = 'Total in Rural (Miles)'
new_table.iat[1, 4] = 'Urban interstate (Miles)'
new_table.iat[1, 5] = 'Urban total (Miles)'
new_table = new_table.drop(0)
new_table.reset_index(drop=True, inplace=True)
new_table.columns = new_table.iloc[0]
new_table = new_table.iloc[1:].reset_index(drop=True)
new_table.to_csv('1980 to 2021_highway.csv', index=False)
time.sleep(4)
os.remove('2021_interstate.xls')

In [ ]:
from selenium.webdriver.common.action_chains import ActionChains

# Create an ActionChains instance
actions = ActionChains(driver)

# JavaScript code to capture click coordinates and send them back to Python
click_capture_script = """
document.addEventListener('click', function(event) {
    var x = event.clientX;
    var y = event.clientY;
    console.log('Clicked at X:', x, 'Y:', y);

    // Send the coordinates back to Python
    window.clickCoordinates = { x: x, y: y };
});
"""

# Execute the JavaScript code in the browser
driver.execute_script(click_capture_script)

# Perform a manual click on the webpage (you can also use Selenium to click on an element within the viewport)
# Example: driver.find_element(By.XPATH, 'your_element_xpath').click()

# Wait for some time to allow for clicks and logging
input('Press Enter to quit...')

# Retrieve the coordinates from JavaScript and print them in Python
click_coordinates = driver.execute_script('return window.clickCoordinates;')
print('Clicked at X:', click_coordinates['x'], 'Y:', click_coordinates['y'])


//*[@id="fullpage"]/table[3]/tbody/tr[4]/td[3]/a 2015
//*[@id="fullpage"]/table[4]/tbody/tr[4]/td[3]/a 2016 
//*[@id="fullpage"]/table[3]/tbody/tr[4]/td[3]/a 2017
//*[@id="fullpage"]/table[3]/tbody/tr[4]/td[3]/a 2018
//*[@id="fullpage"]/table[3]/tbody/tr[4]/td[3]/a 2019
//*[@id="fullpage"]/table[3]/tbody/tr[4]/td[3]/a 2020
//*[@id="fullpage"]/table[3]/tbody/tr[5]/td[3]/a 2021

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.chrome.options import Options
import os
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException
from IPython.display import display, clear_output
import ipywidgets as widgets
import tkinter as tk
from tkinter import filedialog
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton
import sys
import re
import zipfile #extract zip file
import requests
from bs4 import BeautifulSoup
import shutil
table = pd.read_excel('2015_interstate.xls')
keywords = ['year', 'Year']
lowercase_table = table.applymap(lambda x: x.lower() if isinstance(x, str) else x)
mask = lowercase_table.isin(keywords)
row_number = mask.any(axis=1).idxmax()
new_table = table.iloc[row_number:]
new_table.reset_index(drop=True, inplace=True)
new_table = new_table.drop(table.columns[0], axis=1)
new_table.columns = new_table.iloc[0]
new_table = new_table.iloc[1:]
new_table = new_table[new_table.iloc[:, 2] >= 1980]
new_table.reset_index(drop=True, inplace=True)
new_table.rename(columns={new_table.columns[3]: 'VMT Millions'}, inplace=True)
new_table.rename(columns={new_table.columns[4]: 'VMT Actural'}, inplace=True)
new_table.rename(columns={new_table.columns[1]: 'Public road Millage'}, inplace=True)
new_table.to_csv('2015_interstate.csv', index=False)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.chrome.options import Options
import os
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException
from IPython.display import display, clear_output
import ipywidgets as widgets
import tkinter as tk
from tkinter import filedialog
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton
import sys
import re
import zipfile #extract zip file
import requests
from bs4 import BeautifulSoup
import shutil
table = pd.read_excel('2021_interstate.xls')
rows_to_search = [6, 7]
filtered_columns = []
keywords = ['Interstate', 'Total', 'Year']
for row in rows_to_search:
        filtered_columns += [col for col in table.columns if any(keyword.lower() in str(table.iloc[row][col]).lower() for keyword in keywords)]
new_table = table[filtered_columns]
new_table.reset_index(drop=True, inplace=True)
new_table = new_table.iloc[6:]
new_table = new_table.drop(8)
new_table.reset_index(drop=True, inplace=True)
new_table.iat[1, 0] = 'YEAR'
new_table.iat[1, 1] = 'Total rural and urban (Miles)'
new_table.iat[1, 2] = 'Rural interstate (Miles)'
new_table.iat[1, 3] = 'Total in Rural (Miles)'
new_table.iat[1, 4] = 'Urban interstate (Miles)'
new_table.iat[1, 5] = 'Urban total (Miles)'
new_table = new_table.drop(0)
new_table.reset_index(drop=True, inplace=True)
new_table.columns = new_table.iloc[0]
new_table = new_table.iloc[1:].reset_index(drop=True)
new_table.to_csv('1980 to 2021_interstate.csv', index=False)
time.sleep(4)
os.remove('2021_interstate.xls')

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.chrome.options import Options
import os
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException
from IPython.display import display, clear_output
import ipywidgets as widgets
import tkinter as tk
from tkinter import filedialog
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton
import sys
import re
import zipfile #extract zip file
import requests
from bs4 import BeautifulSoup
import shutil
table = pd.read_excel('2017_interstate.xls')
table

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.chrome.options import Options
import os
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException
from IPython.display import display, clear_output
import ipywidgets as widgets
import tkinter as tk
from tkinter import filedialog
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton
import sys
import re
import zipfile #extract zip file
import requests
from bs4 import BeautifulSoup
import shutil
download_path = "/Users/frrookie521/Desktop/civil/cee495/"
years = range(2015, 2022)
for year in years:
    excel_file = os.path.join(download_path, f"{year}_interstate.xls")
    csv_file = os.path.join(download_path, f"{year}_interstate.csv" )
    table = pd.read_excel(excel_file, engine='xlrd')     
    keywords = ['year', 'Year']
    lowercase_table = table.applymap(lambda x: x.lower() if isinstance(x, str) else x)
    mask = lowercase_table.isin(keywords) #create true or false for the keyword row 
    row_number = mask.any(axis=1).idxmax()
    new_table = table.iloc[row_number:]
    new_table.reset_index(drop=True, inplace=True)
    new_table = new_table.drop(table.columns[0], axis=1)
    new_table.columns = new_table.iloc[0]
    new_table = new_table.iloc[1:]
    new_table.iloc[:, 2] = pd.to_numeric(new_table.iloc[:, 2], errors='coerce') 
    new_table = new_table[new_table.iloc[:, 2] >= 1980]
    new_table.reset_index(drop=True, inplace=True)
    new_table.rename(columns={new_table.columns[3]: 'VMT Millions'}, inplace=True)
    new_table.rename(columns={new_table.columns[4]: 'VMT Actural'}, inplace=True)
    new_table.rename(columns={new_table.columns[1]: 'Public road Millage'}, inplace=True)
    new_table.to_csv(csv_file, index=False)